# 목차 

<p align="center"><img src = "https://drive.google.com/uc?id=1E6JrpGk0nJU0cbQq4M4lOJQn2J2XpWP8">

# 개발 환경 구성

In [ ]:
# _1. Package load ------------------------------------------------------------
suppressPackageStartupMessages(library(plotly) )
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(tidyquant))
suppressPackageStartupMessages(library(magrittr))
#suppressPackageStartupMessages(library(inspectdf) )
#suppressPackageStartupMessages(library(tidymodels) )
suppressPackageStartupMessages(library(cowplot) )
suppressPackageStartupMessages(library(timetk) )
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(tictoc) )
suppressPackageStartupMessages(library(lubridate))
suppressPackageStartupMessages(library(collapse))
suppressPackageStartupMessages(library(ggTimeSeries))
suppressPackageStartupMessages(library(fmsb))
#devtools::install_github("ricardo-bion/ggradar",dependencies = TRUE)
suppressPackageStartupMessages(library(ggradar))
suppressPackageStartupMessages(library(DBI))
suppressPackageStartupMessages(library(RSQLite))
suppressPackageStartupMessages(library(zeallot))
suppressPackageStartupMessages(library(furrr))
suppressPackageStartupMessages(library(echarts4r))
suppressPackageStartupMessages(library(gt))
 


# 최종 구현된 사용자인터페이스

**조건 입력 화면**
<p align="center"><img src = "https://drive.google.com/uc?id=1f5LkJpvilncBGK9540UquzdwFClFFv4V">

**초기 화면 상단**
<p align="center"><img src = "https://drive.google.com/uc?id=1YcyH51VR-RVSnDGaEi81oKt7UP8gVJVt">

**초기 화면 하단**

<p align="center"><img src = "https://drive.google.com/uc?export=view&id=1HyP3qeHElEIBc0Ly8uZZkE1RGJ3MGwZY">

**전략탐색화면 상단** 


<p align="center"><img src = "https://drive.google.com/uc?id=1oXLI1NY22N8hlhDrzpfqb8rpSCfO3_kQ">

**전략탐색화면 하단**

<p align="center"><img src = "https://drive.google.com/uc?id=1jWomVVNrSYOAF6JTsNAp3RYCGuExS7af">

# 제 1장. 서론

### 1-1. 배경 및 목적

주식 시장에 처음 뛰어든 초보들은 자신만의 투자 전략을 세우는 것이 쉬운 일이 아닙니다. PER, 유동성, 변동성 등의 지표도 어려울 뿐만 아니라, 금리나 환율과 같은 거시경제지표로 경제 시황을 분석해 이것이 주식시장에 미칠 영향까지 고려하는 것은 더더욱 어려운 일입니다.
따라서 초보 투자자들은 자신만의 전략을 세우지 못하고 남들을 따라하거나 휩쓸려서 거래하는 뇌동매매를 하기 쉽습니다.
이런 초보 투자자들이 쉽게 자신의 전략을 세울 수 있도록 돕고자, ‘백테스팅 툴 – 퀀트롤(Quant + Control)’을 고안하였습니다. 저희는 거시지표와 온라인 뉴스 지표를 활용해 투자일을 선정하고, 거래정보와 기업정보를 활용해 투자할 종목을 선정하고, 직관적인 지표를 반환하여 사용자가 투자전략을 점검하고 비교할 수 있도록 하고자 합니다.

### 1-2. 로드맵

<p align="center"><img src = "https://drive.google.com/uc?id=128QfCoaU0Za4ZW6pptPwOO9wtg731czh">

# 제 2장. 연구구성

### 2-1. 데이터 목록 및 설명

구글드라이브 다운로드 링크(https://drive.google.com/drive/folders/15iKw3AjQnCog0ZEfzGQu2LVc5Qp4X3JO?usp=sharing)

 (1) trading_2019_to_2022.csv
  - 출처 : KRX 정보데이터 시스템(http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201)
  - 포함된 정보 : 일별 주식 거래 정보(2019년 1월 ~ 2022년 6월)
  - 데이터 처리과정 :

      1.전종목기본정보 
      
      2.전종목 시세 
      
      3.PER/PBR/배당수익률 
      
      4.외국인보유량
      
      5.업종분류현황 

   를 각각 크롤링 후 종목코드와 종목명으로 데이터 결합. 이때 우선주, 스팩주, KONEX시장 종목은 제외하였습니다.

  - 변수명 : 
     종목코드, 종목명, 시장구분, 소속부, 종가, 대비, 등락률, 시가, 고가, 저가,
     거래량, 거래대금, 시가총액, 상장주식수, 상장일, EPS, PER, BPS, PBR, 
     주당배당금, 배당수익률, 외국인보유수량, 외국인지분율, 외국인한도수량, 외국인한도소진율, 업종명, open_date 
     
     다양한 정보를 결합하여 종목거래정보에 대한 추가적인 조건을 설정할 수 있습니다. 
     현재는 간단하게 시가총액, PER, 유동성, 변동성을 기준으로 조건을 설정할 수 있게 했습니다.PER은 시가총액을 연간 순이익으로 나눈 것이기 때문에 이 값이 낮은 것이 높은 것보다 저평가 되었다고 생각할 수 있습니다. 하지만 이익이 - 일 경우 값이 NA로 채워져 있었고, 이에 대한 처리를 하기 위해 NA가 아닌 것에 역수를 취한뒤, NA인 값에 0을 대입하여 역 PER변수를 생성하였습니다. 따라서 이 역 PER은 높을수록 다른 종목에 비해서 저평가 되었다고 생각할 수 있습니다. 유동성은 거래량/상장주식수이고 , 변동성은 직전 30거래일의 거래량 평균을 당일 거래량으로 나눠서 평소와 다른 경향이 있는지를 확인하고자 했습니다.
  이후 각 변수별로 일별 rank를 구한 뒤, rank의 percentile을 반환하는 함수를 정의하여 사용자가 임의로 특정 quantile 을 지정하여 필터링 할 수 있게 하였습니다.


 (2) macro_filter.csv
  -  출처 : 한국은행 경제통계시스템 ECOS(https://ecos.bok.or.kr)
  -  포함된 정보 : 거시경제지표 및 온라인뉴스지표의 모멘텀
  -  변수명 : 콜금리, 회사채수익률, 원/달러 환율, 원/위안 환율, 소비자물가지수, 국제유가, 뉴스심리지수 , 금
  -  거시지표 중 7가지를 선택한 이유(부록 참고)
  -  데이터 처리과정 : 
  
  한국은행 ECOS API를 이용하여 
  100대 통계지표 중 콜금리, 회사채수익률, 원/달러,원/위안 환율, 소비자물가지수, 국제유가,금  2022.02.08부터 공개된 실험적 통계인 뉴스심리지수 정보를 가져올 수 있었습니다.
  
  **뉴스심리지수(NSI)는 인터넷 포털사이트의 경제분야 뉴스기사를 
   기반으로 표본문장을 추출한 뒤 각 문장에 나타난 긍정, 부정,
   중립의 감성을 기계학습(machine learning) 방법으로 분류하고 긍정과
   부정 문장수의 차이를 계산하여 지수화한 지표
  
  
  변수별 데이터 업데이트 주기:
  
  콜금리,회사채수익률,원/달러환율,원/위안활율 산출 주기: 일별
  
  소비자물가지수, 국제유가, 금 산출 주기 : 월별 
  
  뉴스심리지수 산출 주기 : 주별 (매주 화요일 오후4시)
  
  모멘텀 산출 방식: 
  
  여기서 모멘텀은 주식시장에서 사용하는 모멘텀이 아닌, 단순히 이전 발표값 대비 + 인지, - 인지 구분하는 변수입니다.
  
ex. 소비자물가지수는 매월 말에 발표합니다. 하지만 매월 마지막주 어떤요일에 발표한다는 내용이 없어서 이 정보는 그 다음달 초부터 사용할수 있다고 처리하였습니다. 예를 들어 1) 5월 소비자물가지수,2) 6월 소비자 물가 지수가 있을때, 7월 1일부터 7월 30일까지 계산된 소비자물가지수 모멘텀은 2) - 1) 의 값의 부호로 처리했습니다.

마찬가지로 뉴스심리지수는 매주 화요일 오후 4시에 (직전 주 화요일) ~ (발표하는 주 월요일)에 대해 발표하고 있습니다. 따라서 뉴스심리지수의 모멘텀은, 첫번째주 월요일과 두번째주 월요일의 모멘텀의 차이를 두번째 주 화요일 오후 4시에 계산 할 수 있고, 그렇게 계산된 모멘텀 정보는 그 주 수요일부터 그다음주 화요일까지 일괄적으로 적용됩니다. 

만약 그 주기별로 이전 발표 값와 새로운 발표 시점의 값이 정확히 일치하면 아직 변동이 없는것으로 간주하고 이전에 발표했을때 계산한 모멘텀으로 다음 발표시점까지 채웠습니다. 


 (3) esg_table.csv
  - 출처 : 한국기업지배구조원 (http://www.cgs.or.kr)
  - 포함된 정보 : 2021년 기준 측정된 기업별 ESG등급
  
아직 확실하게 정착된 개념이 아니고 평가방법도 매년 조금씩 달라져서 활용하기 어렵지만 앞으로 활용할 수 있는 정보라 생각하여 
참고 지표로 선택할 수 있게 결합하였습니다.평가지표 활용에 대해 <esgdata@cgs.or.kr>에 사전 문의해본 결과 데이터 출처 표기 후 사용해도 된다는 허가를 받았습니다.

변수를 계산하는 과정에서 맨 앞 날짜에 대한 직전30거래일에 대한 정보, 이전 두 달 값에 대한 moment 는 구할 수 없어서 NA처리가 되었고, 따라서 넉넉하게 NA가 없는 
2019-07-01 ~ 2022-06-30 3년간의 온전한 데이터를 사용하였습니다.

 (4) KOSPI_KOSDAQ_2019_2022.csv
  -  출처 : 한국은행 경제통계시스템 ECOS(https://ecos.bok.or.kr)
  -  포함된 정보 : 코스피, 코스닥 일별 종가
  -  변수명 :코스피지수(1980.01.04=100)	코스닥지수(1996.07.01=1000)
  
  설정한 전략의 기대수익률과, 관찰기간동안의 지수 등락률을 비교하여 시장대비 승률을 계산하는데에 사용하였습니다.
 
(5) iljung_variable.db
 - 본선 기간 중 제공받은 일중거래데이터로 새로운 종목필터링 변수(일중 수익률 변동성) 가공
 
(6) daily_and_iljung.db
 - 본선 기간 중 제공받은 일별 데이터의 기준가를 통해서 계산한 수정주가를 기존 trading_2019_to_2022.csv 와 결합하기 위해서 12자리 종목코드변수가 필요했고, 일중데이터를 통해 결합
 
(7) krx_2019_2022.db
- 본선 기간 중 제공받은 일중데이터를 한번에 메모리에 올려쓸 수 없기 때문에 데이터베이스 파일로 저장.

(8) new_variable_daily_fluc.rds
- 일중 수익률 변동성을 가공하여 저장해둔 R 변수 파일.

(9) new_variable_with_adjusted_price.rds
- 일중 수익률 변동성변수에 수정주가까지 계산한 R 변수 파일

(10) back_test_chance=0.1,minus=-0.05,a=-0.01,obs_period=30.rds
- 목표수익률 10%,
  최대손실율 -5%,
  목표구매가 -1%,
  관찰기간 30일 을 넣었을때 2019년7월1일~2022년6월30일 까지 모든 데이터에 대해 back_test함수를 적용한 R 변수 파일.

In [ ]:
# https://drive.google.com/drive/folders/1nRCGZTLcI5vvZmTXmgNimf0pyVtOBla4?usp=sharing

n_day_data<- fread("C:/Users/shj/Documents/2022_summer_project/krx_repo/trading_2019_to_2022.csv",encoding = "UTF-8")
n_day_data %>% 
  fmutate(종목코드 = sprintf("%06d",종목코드) ) %>% 
  left_join(readRDS(file = "C:/Users/shj/Documents/2022_summer_project/krx_repo/new_variable_daily_fluc.rds"),
            by = c("종목코드","open_date"="거래일자")) %>% 
  fmutate(open_date = ymd(open_date)) ->n_day_data
#n_day_data  <- read_rds("C:/Users/shj/Documents/2022_summer_project/krx_repo/new_variable_with_adjusted_price.rds")
# 이 파일이 기존의 trading_2019_2022.csv에 일중데이터 결합 및 수정주가를 결합한 것이다. 하지만 일중데이터db(krx_2019_2022.db)에 저장할때도 수정 주가를 
# 반영해야 하는데 반영하지 못했으므로 일단 인지해놓은 상태로 수정하지 않은 주가로 진행
# 전처리 과정은 부록참조.

n_day_data %>% 
  fmutate(open_date = ymd(open_date)) ->>n_day_data

macro_filter <- fread("C:/Users/shj/Documents/2022_summer_project/krx_repo/macro_filter.csv",encoding = "UTF-8")
macro_filter %>% tail(30)
esg.data <- fread("C:/Users/shj/Documents/2022_summer_project/krx_repo/esg_table.csv",encoding = "UTF-8")
esg.data %>% 
  select(2,4) %>%
  filter(nchar(ESG등급)>0)->esg.data

esg.data %>% head()
# ESG 합치기
n_day_data <- merge.data.table(n_day_data, esg.data, by.x="종목명", by.y="기업명", all.x=T)

n_day_data %<>% 
  fmutate(ESG등급 = ifelse(is.na(ESG등급),"미지정",ESG등급) %>%as_factor()) %>% 
  fmutate(ESG등급 = ESG등급 %>% fct_relevel("A+","A","B+","B","C","D","미지정") )
  
n_day_data$ESG등급 %>% levels()
# ECOS 데이터 합치기

n_day_data <-merge.data.table(n_day_data, macro_filter, by.x="open_date", by.y="time", all.x=T)


n_day_data %>% tail()

time,콜금리(익일물)(연%) _momentum,"회사채수익률(3년,AA-)(연%) _momentum",원/달러 환율(종가)(원) _momentum,원/위안 환율(종가)(원) _momentum,소비자물가지수(2020=100) _momentum,"국제유가(Dubai, 현물)(U$/bbl) _momentum",금($/OZ) _momentum,뉴스심리지수() _momentum
<date>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
2022-07-02,1,1,-1,1,1,-1,-1,1
2022-07-03,1,1,-1,1,1,-1,-1,1
2022-07-04,1,-1,-1,-1,1,-1,-1,1
2022-07-05,-1,-1,-1,1,1,-1,-1,1
2022-07-06,-1,-1,1,1,1,-1,-1,-1
2022-07-07,1,-1,1,1,1,-1,-1,-1
2022-07-08,1,1,-1,-1,1,-1,-1,-1
2022-07-09,1,1,-1,-1,1,-1,-1,-1
2022-07-10,1,1,-1,-1,1,-1,-1,-1


기업명,ESG등급
<chr>,<chr>
AJ네트웍스,B
AK홀딩스,B+
AP시스템,D
BGF,A
BGF리테일,A
BNK금융지주,A+


[1] "A+"     "A"      "B+"     "B"      "C"      "D"      "미지정"

open_date,종목명,종목코드,시장구분,소속부,종가,대비,등락률,시가,고가,⋯,일중수익률변동성,ESG등급,콜금리(익일물)(연%) _momentum,"회사채수익률(3년,AA-)(연%) _momentum",원/달러 환율(종가)(원) _momentum,원/위안 환율(종가)(원) _momentum,소비자물가지수(2020=100) _momentum,"국제유가(Dubai, 현물)(U$/bbl) _momentum",금($/OZ) _momentum,뉴스심리지수() _momentum
<date>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<int>,<int>,⋯,<dbl>,<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
2022-06-30,흥국,010240,KOSDAQ,우량기업부,6850,-70,-1.01,6990,7220,⋯,0.013076289,미지정,1,1,1,1,1,1,-1,1
2022-06-30,흥국에프엔비,189980,KOSDAQ,우량기업부,2870,-115,-3.85,3000,3005,⋯,0.010486763,미지정,1,1,1,1,1,1,-1,1
2022-06-30,흥국화재,000540,KOSPI,NA,3245,-45,-1.37,3270,3295,⋯,0.004126549,B+,1,1,1,1,1,1,-1,1
2022-06-30,흥아해운,003280,KOSPI,NA,2080,-85,-3.93,2250,2250,⋯,0.010175625,D,1,1,1,1,1,1,-1,1
2022-06-30,희림,037440,KOSDAQ,우량기업부,8870,1330,17.64,7950,9630,⋯,0.036852668,미지정,1,1,1,1,1,1,-1,1
2022-06-30,힘스,238490,KOSDAQ,우량기업부,6270,30,0.48,6260,6270,⋯,0.004562113,미지정,1,1,1,1,1,1,-1,1


## 2-2. 전처리 과정



실제 매수가격, 매도가격을 반영하기 위해 현재 시장별 호가제도를 참고하여 tick단위로 조정하였습니다.
<p align="center"><img src = "https://drive.google.com/uc?id=1hiLT8r7RB2ZNMCXoL4LjNMWpm4F241jC">


In [ ]:
tick_size_adjust<- function(x,y,type = "buy"){
  z <- nchar(y) 
  
  # KOSPI ,KOSDAQ주식 가격별 최소단위호가
  tick_min <-  fifelse(
    (x >= 500000 &z==5), 1000,# KOSPI
    fifelse(x >= 100000 &z==5 , 500,
            fifelse(x >=  50000 ,  100, # KOSPI & KOSDAQ 공통
                    fifelse(x >=  10000 ,   50,
                            fifelse(x >=   5000 ,   10,
                                    fifelse(x >=   1000 ,    5,
                                            fifelse(TRUE , 1,1 )))))))
  
  # 구매할때(buy)는 가장 가까운 밑에 호가,
  # 익절할때(!buy)는 가장 가까운 위에 호가로 지정.
  
  ifelse(type == "buy",
         x <- x-x%%tick_min,
         x <- fifelse(x%%tick_min ==0,x,x-x%%tick_min+tick_min)
  )
  
  return(x)
}
n_day_data %>% filter(거래량>0) %>% sample_n(100) %>% 
select(open_date,종목명,시장구분,종가) %>% 
mutate(목표익절가 = 종가*1.05) %>% 
mutate(목표익절가_tick_adjust_when_sell = tick_size_adjust(종가*1.05,시장구분,type = "sell")) %>% 
mutate(목표구매가 = 종가*0.98) %>% 
mutate(목표구매가_tick_adjust_when_buy = tick_size_adjust(종가*0.98,시장구분,type = "buy"))  %>% head(10)

# 거래비용으로 인한 오차를 줄이기 위해서, 구매할때는 가장 가까운 밑에호가로 걸어두고
# 판매할 때는 가장 위에 호가에 걸어둬서 싸게구매, 비싸게 판매하는 조건으로 정했다.


open_date,종목명,시장구분,종가,목표익절가,목표익절가_tick_adjust_when_sell,목표구매가,목표구매가_tick_adjust_when_buy
<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
2020-04-21,서전기전,KOSDAQ,3195,3354.75,3355,3131.1,3130
2020-04-20,토탈소프트,KOSDAQ,3005,3155.25,3160,2944.9,2940
2020-07-08,빅텍,KOSDAQ,7150,7507.50,7510,7007.0,7000
2019-08-09,SK바이오랜드,KOSDAQ,12650,13282.50,13300,12397.0,12350
2020-03-04,삼화왕관,KOSPI,40650,42682.50,42700,39837.0,39800
2019-05-28,잇츠한불,KOSPI,22750,23887.50,23900,22295.0,22250
2020-08-12,광진윈텍,KOSDAQ,2995,3144.75,3145,2935.1,2935
2021-06-22,바이오톡스텍,KOSDAQ,10950,11497.50,11500,10731.0,10700
2019-11-12,비상교육,KOSPI,8340,8757.00,8760,8173.2,8170


시가총액, PER, 유동성, 변동성 변수를 정의하고 계산한 변수들의 percentile을 구하여 추후 종목필터링에 활용하였습니다.

In [ ]:
return_rank_percentile<- function(x,invesrse = FALSE){### 사전 정의함수 1.rank percentile 변환
  if(invesrse == TRUE){
    x <- -x
  }
  x <- (frank(x,ties.method ="min")/length(x))*100 # 작은값일수록 순위가 높음. (= 최소값이 1등)
  return(x)
}


n_day_data |>
  # 변수만드는데 필요한 계산 먼저 완료.
  fmutate(시가총액 = 시가총액 / 100000000)  |>  #overflow문제있는것같아서 억단위로 바꿈.결과에는 영향 없음.
  fgroup_by(종목코드)  |> 
  fmutate(거래량_lag = flag(거래량,1))  |> 
  fmutate(고가_lag = flag(고가, 1))  |> 
  fmutate(roll_mean_30_거래= frollmean(x = 거래량_lag,n = 30,align = "right",fill = 1) )  |>  
  fmutate(`거래빈도 기준 거래속도(30거래일)`= 거래량/roll_mean_30_거래)  |>  
  fmutate(`최근20거래일고가` = frollapply(고가_lag, 20, max))  |> 
  fmutate(`20거래일고가대비등락률` = (종가-최근20거래일고가)/종가) |>  
  fungroup() |>
  fgroup_by(open_date) |>
  fmutate(new_PER = ifelse(is.na(PER),0,1/PER)) |>
  fmutate(유동성 = 거래량/상장주식수) |>
  
  # 계산된 변수들에 percentile 반환하는 함수 일괄 적용
  fmutate(across(.cols = c(시가총액,new_PER,유동성,`거래빈도 기준 거래속도(30거래일)`, `20거래일고가대비등락률`,일중수익률변동성),
                 .fns = function(x)return_rank_percentile(x,invesrse = TRUE),
                 .names = function(.cols,f) paste0(.cols,"_percentile") )) |>
  fungroup() %>%
  fselect(names(.) %>% grep(pattern ="open_date|종목|종가|거래량|소속부|시장구분|momentum|_percentile|ESG")) |>
  # 앞에 NA 자르고, 2022-06-30 까지 있으니까 현재 기준으로 필터링은 되지만 아직 탈출률 결정안된 날들이 있음
  # 일중데이터를 통한 보완은 1분기밖에 없어서 못했다고하면 될듯.
  # 깔끔하게 2019-07-01 ~ 2022-06-30 3년데이터
  filter(open_date >="2019-07-01") ->  processed_data

processed_data %>% head()

open_date,종목명,종목코드,시장구분,소속부,종가,거래량,ESG등급,콜금리(익일물)(연%) _momentum,"회사채수익률(3년,AA-)(연%) _momentum",⋯,"국제유가(Dubai, 현물)(U$/bbl) _momentum",금($/OZ) _momentum,뉴스심리지수() _momentum,거래량_lag,시가총액_percentile,new_PER_percentile,유동성_percentile,거래빈도 기준 거래속도(30거래일)_percentile,20거래일고가대비등락률_percentile,일중수익률변동성_percentile
<date>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<fct>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2019-07-01,3S,060310,KOSDAQ,중견기업부,2700,138345,미지정,1,-1,⋯,1,1,1,107811,50.79213,65.86654,55.97696,49.11186,89.82237,9.169467
2019-07-01,AJ네트웍스,095570,KOSPI,NA,5320,51445,B,1,-1,⋯,1,1,1,138345,28.85262,3.31253,78.54057,61.83389,34.08545,73.451752
2019-07-01,AJ렌터카,068400,KOSPI,NA,11050,238185,미지정,1,-1,⋯,1,1,1,51445,29.23668,58.32933,25.06001,22.70763,33.26932,8.689390
2019-07-01,AK홀딩스,006840,KOSPI,NA,44500,20570,B+,1,-1,⋯,1,1,1,238185,14.06625,4.75276,72.29957,62.36198,19.77916,35.717715
2019-07-01,APS홀딩스,054620,KOSDAQ,우량기업부,5760,370566,미지정,1,-1,⋯,1,1,1,20570,51.89630,60.58569,16.08257,14.69035,88.28613,14.354297
2019-07-01,AP시스템,265520,KOSDAQ,우량기업부,26150,114800,D,1,-1,⋯,1,1,1,370566,20.69131,28.94863,30.53289,63.99424,18.77100,35.045607


# 제 3장. 투자전략 설명

특정 투자 시작일을 정해서 단 한 번만 매입 후 관찰하다가 특정 조건을 만족했을 때 무조건 매도하는 전략을 활용하기로 하였습니다. 구체적인 투자 시작일 선정 방법, 투자 종목 선정 방법, 매입 전략, 매도 전략은 아래와 같습니다.

### 3-1. 투자 시작일 선정 방법
 거시지표 및 온라인 뉴스 지표의 모멘텀을 활용하여 투자 시작 일자를 선정합니다. 콜금리, 회사채수익률, 원/달러 환율, 소비자물가지수, 국제유가, 뉴스심리지수, 원/위안 환율, 금의 8가지 지표를 선정하여 거시지표 및 온라인 뉴스 지표로 활용하였습니다.

 구체적인 투자 시작일 선정 방식은 다음과 같습니다. 8가지 지표의 모멘텀을 각각 “+”, “-” 로 선택하여 전략에 활용하고자 하는 경제 시황을 8가지 지표의 “+”, “-” 조합을 통해 나타냈습니다. 8가지 지표의 “+”, “-” 조합이 나타내는 날을 투자시작일로 정하고 투자를 시작합니다.


### 3-2. 투자 종목 선정 방법
  먼저 어떠한 종목을 골라서 매입할 것인지 결정해야 합니다. 관찰 기간 하루 전날의 거래정보 및 기업정보를 활용하여 종목을 고를 것이고 KOSPI, KOSDAQ 시장에 상장된 전체 종목을 대상으로 매입할 종목들을 선택할 것입니다.

 거래정보 및 기업정보를 잘 대표 할 수 있다고 생각되는 지표를 일곱 개 선정하여 종목 필터링 지표를 구성하였고 ESG 등급, 역 PER, 시가총액, 유동성, 변동성, 거래빈도 기준 거래속도, 20거래일 고가 대비 등락률을 지표로 활용하기로 하였습니다. 이때, 유동성 지표는 “거래량/상장 주식 수", 변동성 지표는 “1분단위 수익률의 표준편차", 거래빈도 기준 거래속도는 "당일 거래량 / 당일 제외 과거 30일 거래일 거래량평균”, 20 거래일 고가 대비 등락율은 "당일 제외 20거래일 기준 고가 대비 당일 종가의 수익률"으로 정의하기로 하였습니다.
 
 구체적인 필터링 방식은 종목별로 약간의 차이가 있습니다. 역PER, 시가총액, 유동성, 변동성, 거래빈도 기준 거래속도, 20거래일 고가대비 등락율에 대해서는 상위 20~50%와 같이 원하는 구간을 percentile 형식으로 선택하는 방식으로 종목을 필터링하였습니다. ESG 등급에 대해서는 A+, A, B+, B, C+, C, 미지정 중에서 원하는 등급을 선택하는 방식으로 종목을 필터링하였습니다.

### 3-3. 매수 전략 
 선택된 종목들을 구매하는 방법에 대한 구체적인 방법을 결정합니다. 종목 필터링이 진행된 다음 날부터 구매를 시작할 수 있게 하였고 종목 필터링이 완료된 날의 종가보다 a% 낮은 가격에서 구매를 진행하기로 하였습니다. 종목 필터링이 완료된 날 다음날부터 3일 후까지 구매 여부를 결정하고 결과적으로 구매가 결정된 종목들에 투자합니다. 


### 3-4. 매도 전략
 먼저 매도 전략을 설명하는데 탈출이라는 용어를 정의해서 활용하고자 합니다. 정해진 목표 수익률을 달성하여 매도하는 것을 탈출이라고 정의하였습니다.

 구매한 종목들은 특별한 매도 조건을 달성하지 못하면 정해진 관찰기간 동안 보유합니다. 그리고 관찰기간 중 매도하는 경우는 크게 두 가지가 있습니다. 첫번째는 정해진 목표 수익률을 달성하면 탈출하여 익절하는 것입니다. 두 번째는 정해진 최대손실률 이상의 손실이 발생하면 손절하는 것입니다.

 전략이 성공적이었는지 판단하는 지표는 4가지를 활용합니다. 첫번째로는 구매한 종목 전체에 대한 탈출한 종목의 비율을 탈출률이라고 정의합니다. 탈출률이 높을수록 좋은 전략임을 의미합니다. 두 번째로는 구매한 종목 전체에 대한 손절한 종목의 비율을 손절률이라고 정의합니다.손절률이 낮을수록 좋은 전략임을 의미합니다. 세 번째로는 구매한 종목들의 수익률들을 평균한 값을 기대수익률이라고 정의합니다. 기대수익률이 높을수록 좋은 전략임을 의미합니다. 네 번째로는 탈출률의 표준편차를 탈출률의 변동성이라고 정의합니다. 탈출률 변동성 대비 탈출률이 높은 전략은 안정적으로 원하는 탈출률을 달성할 수 있음을 의미합니다. 

# 제 4장. 전략 검증 과정

### 4-1. 전략 검증 방법 개요

 3장에서 설명한 투자전략을 과거에서 검증하는 백테스팅을 실행하고 그 결과를 백테스팅 툴 사용자에게 제공하여 앞으로의 투자에 참고가 될 수 있도록 합니다. 구체적인 전략 검증 방법은 아래와 같습니다.

1. 과거 데이터가 있는 날짜 중 전략검증기간을 설정하여 전략검증기간 동안 3장에서 정의한 투자전략에 대해서 관찰하도록 지정합니다.

2. 전략검증기간 중에서 투자전략에서 정한 거시변수 조합과 일치하는 투자 시작일을 모두 골라냅니다.

3. 각 투자 시작일에서 종목 필터링을 시행한 후 관찰 기간 동안 전략을 수행하여 탈출률, 손절률, 기대수익률 등 전략이 성공적이었는지 판단해주는 지표들을 산출합니다.

4. 각 날짜별로 구해진 탈출률, 손절률, 기대수익률 등을 평균하여 평균탈출률, 평균손절률, 평균기대수익률을 구하고 벤치마크와 비교하여 전략이 성공적이었는지 판단합니다.




### 4-2. 전략 검증에 사용될 백테스트 함수

구체적인 전략 검증에 사용할 백테스트 함수를 정의했습니다. 함수의 input과 output을 위주로 간략히 설명드리자면,

1. 함수의 input으로 목표 수익률, 최대 손실률, 심리적 저항선 비율, 관찰 일수, 구매 결정 퍼센트를 받습니다. 각각의 정의는 다음과 같습니다.


*   목표 수익률 (chance) : 구매 가격 대비 몇 퍼센트의 수익이 나면 탈출할지를 지정합니다.
*   최대 손실률 (minus) : 구매 가격 대비 몇 퍼센트의 손실이 나면 손절할지를 지정합니다.
*   심리적 저항선 비율 (sad) : 구매 가격 대비 일정 퍼센트 이상의 손실이 나면 손절은 하지는 않되, 손절의 압박을 느낄 수 있습니다. 해당 퍼센트를 정해주어 관찰 기간동안 손절의 압박을 느낀 횟수를 계산 해줍니다. 심리적 저항선 비율은 최대 손실률보다 클 수 없으며, 심리적 저항선에 도달하기 전 탈출을 하는 경우에는 횟수를 계산하지 않습니다.
*   관찰 일수 (obs_period) :필터링된 날짜로부터 며칠을 관찰할 것인지를 지정합니다. 관찰 일수 내에 손절도 탈출도 하지 않은 경우 관찰 일수는 보유기간이 됩니다.
*   구매 결정 % (a) : 주가가 필터링된 날짜의 (1-a)\*(전날 종가)에 도달하면
해당 주식을 구매합니다. 필터링된 날짜 기준으로 3일 내에 (1-a)*(전날 종가)에 도달하지 않으면 구매하지 않는 것으로 간주합니다.
*   end_date : 미래참조편향을 고려하여 설정 기간 마지막날 기준으로 탈출률이 계산되지 않는 항목을 삭제해주고자 합니다. 따라서 설정 기간의 마지막날을 자동으로 함수에 넣을 수 있도록 하였습니다.

2. output으로는 날짜별 선택된 종목에 대한 정보(obs_list)를 반환합니다. 


*   날짜별 선택된 종목에 대한 정보 (obs_list) : back_test 함수 구현 과정에서 필요한 변수들과 종목에 대한 정보가 담긴 데이터입니다. 종목명, 종가, open_date (필터링된 날짜), obs_expired (필터링된 날짜+관찰 기간), 구매 가격, 구매 (구매 여부), sad_count (심리적 저항 횟수) 등이 포함되어 있습니다. obs_list을 이용하여 5장에서 탈출률, 손절률 등을 구할 예정입니다.

구체적인 구현 방식은 주석으로 덧붙였습니다.



In [ ]:

back_test_final<- function(data = data,
                           chance = 0.05, # 목표 수익률
                           minus = -0.10, # 최대 손실률: 손절가 느낌
                           sad = -0.05, # 심리적 저항선 비율
                           obs_period = 30,  # 관찰 일 수
                           a = -0.01){ # 구매 결정 %
 
  data |>
    rename(filtered_date = open_date)|>
    fselect(종목명,종목코드,시장구분,종가,filtered_date )|>
    fmutate(obs_expired = filtered_date + obs_period) ->obs_list 


  gc()
 
  
  print("분석을 시작합니다.")
  
  tic()
  
  obs_list |>
    rename(filtereddate_종가 = 종가) |>
    fmutate(open_date =furrr::future_map2(filtered_date, obs_expired, `:`), # 관찰일로부터 관찰기간까지의 리스트를 만들고
            
            목표구매가 = filtereddate_종가 * (1+a), # 설정 조건들(chance, minus, a)에 맞게 변수 생성.
            실제목표구매가 = tick_size_adjust(목표구매가,시장구분,type = "buy"),
            실제목표익절가 = tick_size_adjust(실제목표구매가*(1+chance),시장구분,type = "sell"),
            실제목표손절가 = tick_size_adjust(실제목표구매가*(1+minus),시장구분,type = "buy")) |>
    unnest(open_date) |>
    fmutate(open_date = as_date(open_date)) %>% 
    # 전체 데이터에서 필요한 정보들을 방금 만들어둔 틀에 맞게 데이터 합치기
    inner_join(.,n_day_data[, .(종목코드,시가, 저가, 고가, 종가,거래량, open_date)],by = c("종목코드","open_date"))|>
    filter(open_date >filtered_date) |>
    fgroup_by(filtered_date,종목코드)|>
    fmutate(
    # 매일매일의 시가는 연속이아니라 15%가격에서 시작할 수도있고, -20%에서 시작할 수도있다. 
    # 우리의 매수, 매도 전략의 가정은 장 시작전에 목표했던 매수가, 매도가에 대해 주문을 걸어두는 것이기 때문에
    # 설정해둔 가격보다 높은 가격에 매도 or
    # 설정해둔 가격보다 낮은 가격에 매수 체결됐다면,실제 체결가에 대한 정보를 저장해야 한다.
    # 매일매일의 시가가 목표 매수,매도 가격을 상회하거나 하회하는 여부를 저장했다.
      observe_구매_gap  = fifelse(거래량 > 0 & 시가 <= 실제목표구매가,1, 0), 
      observe_익절_gap  = fifelse(거래량 > 0 & 시가 >= 실제목표익절가,1, 0),
      observe_손절_gap  = fifelse(거래량 > 0 & 시가 <= 실제목표손절가,1, 0),
        
    # 초기 목표한 매수 , 매도가
      buy_success  = fifelse(거래량 > 0 & 저가 <= 실제목표구매가,1,0),
      u_chance_pct = fifelse(거래량 > 0 & 고가 >= 실제목표익절가,1,0),
      u_minus_pct  = fifelse(거래량 > 0 & 저가 <= 실제목표손절가 * (1+minus),1,0),
        
    # 일별데이터에는 시가 저가 고가 종가가 있을 뿐, 저가와 고가 중 어떤것이 먼저 거래되었는지에 대한 정보가 없어서
    # 탈출을 먼저 했는지, 손절을 먼저 했는지에 대해서 순서확인이 필요한 경우가 있다.
    # 순서확인이 필요한 경우를 필터링하기 위해서 두번째 index까지 저장해둔다.
      탈출_first_idx =ifelse(sum(u_chance_pct)!=0,which.max(u_chance_pct) ,10000),
      손절_first_idx =ifelse(sum(u_minus_pct)!=0,which.max(u_minus_pct) ,10000),
    # 두번째 index는, 만약 구매한날과 첫번째 탈출한 날이 같을때 순서를 확인했더니 탈출가격->구매가격이라고 하자.
    # 이날은 구매만 된것이고, 따라서 첫번째 탈출idx가 아닌 두번째 idx를 확인해야 한다. 그런데 우연히
    # 두번째 탈출 idx와 첫번째 손절 idx가 같은 경우가 있을 수 있기 때문에 둘다 이 경우를 대비하여 다 저장을 해둬야 한다.
      탈출_second_idx = ifelse(sum(u_chance_pct)>=2,which(u_chance_pct==1)[2] ,10000),
      손절_second_idx = ifelse(sum(u_minus_pct)>=2,which(u_minus_pct==1)[2] ,10000),
    # 실제 목표 구매가보다 설정해둔 비율(sad)만큼 낮은가격이 존재하는 날에 대해서 슬픈날로 정의.
      sad_touch    = fifelse(거래량 > 0 & 저가 <= 실제목표구매가*(1+sad), 1,0) ) |>
    
    # 거래일 기준으로 3거래일 이내 구매 성공 여부 구분
    fsummarise(        
        
#    group_by(filtered_date,종목코드) 했기 때문에 시장구분, 종목명은 전부 같은값으로 저장되어있고, 그중 첫번째를 가져오라는 명령어
      시장구분 = 시장구분[1],
      종목명 = 종목명[1],
   
#  매수한날의 시가와 비교해서 gap이 있는 날이면 gap가격으로 반영
      실제구매가 =ifelse(observe_구매_gap[ifelse(length(which(buy_success==1))==0,1 ,min(which(buy_success==1)) )]==1 ,시가[ifelse(length(which(buy_success==1))==0,1 ,min(which(buy_success==1)) )],실제목표구매가),       
      실제익절가_first =ifelse(탈출_first_idx[1]!=10000 & observe_익절_gap[탈출_first_idx[1]]==1 ,시가[탈출_first_idx[1]],실제목표익절가),
      실제익절가_second =ifelse(탈출_second_idx[1]!=10000 & observe_익절_gap[탈출_second_idx[1]]==1 ,시가[탈출_second_idx[1]],실제목표익절가),
      실제손절가_first =ifelse(손절_first_idx[1]!=10000 & observe_손절_gap[손절_first_idx[1]]==1 ,시가[손절_first_idx[1]],실제목표손절가),
#  위에서 이미 계산 한것들이고 group에따라 각각 같은값들이 저장되어있으므로 그중 첫번째 인덱스를 통해 가져옴.        
      목표구매가 =실제목표구매가[1],
      목표익절가 =실제목표익절가[1],
      목표손절가 =실제목표손절가[1],
      
      # 구매성공여부의 합이 0이면 구매 실패.100반환, 구매성공(=1)시 1이 몇번째 인덱스에 나오는지 반환
      buy_n_days =ifelse(sum(buy_success)!=0,which.max(buy_success)  ,100),
      
      # 관찰기간내 저가
      min_price =suppressWarnings(min(저가[which(거래량>0)]))  , 
      
      # 관찰기간내 고가
      max_price = suppressWarnings(max(고가[which(거래량>0)])),
      
      # 관찰기간내 총 거래일 수
      total_trade_days = length(open_date),
      
      # if_else로 바꾸면 안되니 바꾸지말것.
      
      #탈출하는데 걸린 거래일 수(탈출 못하면 10000),및 날짜반환. 
      탈출_first =탈출_first_idx[1],
      탈출_second=탈출_second_idx[1],
      #확인이 필요한 날짜가 언제인지 알기 위해 저장
      탈출_first_date =open_date[탈출_first_idx[1]],
      탈출_second_date = open_date[탈출_second_idx[1]],
      
      #손절하는데 걸린 거래일 수, 손절 못하면 10000
      손절_first =손절_first_idx[1],
      손절_first_date =open_date[손절_first_idx[1]],
      
     # 슬픈 비율
      sad_count = ifelse((sum(u_chance_pct)+sum(u_minus_pct))==0,
                         sum(sad_touch[which.max(buy_success):length(open_date)]),
                         sum(sad_touch[which.max(buy_success):min(탈출_first_idx[1],손절_first_idx[1])])),
        
     # 탈출_second,손절_second변수와 같은 이유로 슬픔날짜 count의 두번째 인덱스를 사용해야 하는 경우가 있으므로 미리 저장해둔다.   
      sad_count_탈second = ifelse((sum(u_chance_pct)+sum(u_minus_pct))==0,
                                 sum(sad_touch[which.max(buy_success):length(open_date)]),
                                 sum(sad_touch[which.max(buy_success):min(탈출_second_idx[1],손절_first_idx[1])])),
      # 관찰마지막날 종가 반환(기대수익률 계산 위함)
      final_price = 종가[length(open_date)],
      
      # 관찰마지막날 거래량반환을 통한 거래정지여부 반환(다양한 이유가 있고 정확히 상장폐지사유면 수익률을 -100으로 반환한다던지 등.)
      # 현재 데이터에는 거래정지 여부만 알 수 있기 때문에 불가능.
      final_volume = 거래량[length(open_date)],
      filtereddate_종가 =filtereddate_종가[1],
      obs_expired =obs_expired[1] 
      
    ) |>
    fungroup() |>
    fmutate(구매 = fifelse(buy_n_days<=3,1,0))->results
  
  print("일별데이터를 통해 1차 분석이 완료되었습니다.")
  print("소요시간:")
  toc()
  gc()
  print("거래순서 확인이 필요한 경우에 대해서 일중데이터를 통해 확인을 시작합니다.")
  
  
  #거래순서 확인이 필요한 경우

#  1. 탈출과 손절이 같은날인 경우 (목표익절가 목표손절가의 순서 확인 필요)  
#  2. 구매와 탈출이 같은날인 경우 (목표구매가와 목표익절가의 순서 확인 필요)  
#  3. 구매와 손절이 같은날인 경우 (구매하자마자 바로 탈출인지, 구매가격이 목표손절가 이하인지 확인 필요)  

  results |>
    filter(구매==1) |> 
    filter( !(탈출_first==10000 & 손절_first ==10000) ) %>% 
    fmutate(across(.cols = names(.) |> grep(pattern = "_date"),.fns = as_date)) |> 
    filter((탈출_first == 손절_first)|(손절_first ==buy_n_days )|(탈출_first)==buy_n_days ) %>% 
    {.->>double_check} |> # 두번 check해야하는 경우가 있다. (밑에 설명 참고)
 #거래순서 확인이 필요한 경우를 필터링 했다.   
# 각 경우별 순서를 확인해야 하는 날짜 저장.
    fmutate(to_check_date = fifelse( (탈출_first == 손절_first)&(손절_first ==buy_n_days),
                                     탈출_first_date ,
                                     fifelse( (탈출_first == 손절_first)&(손절_first !=buy_n_days) ,
                                              탈출_first_date,
                                              fifelse( (탈출_first != buy_n_days)&(손절_first ==buy_n_days),
                                                       손절_first_date     ,
                                                       탈출_first_date   )
                                     ) )) |>
# 각 경우가 어떤 경우인지 볼 수 있게 변수로 저장.
    fmutate(check_case =fifelse( (탈출_first == 손절_first)&(손절_first ==buy_n_days),
                                 "탈=손=구" ,
                                 fifelse( (탈출_first == 손절_first)&(손절_first !=buy_n_days) ,
                                          "탈=손!=구",
                                          fifelse( (탈출_first != buy_n_days)&(손절_first ==buy_n_days),
                                                   "탈!=구=손"     ,
                                                   "탈=구!=손"   )
                                 ) )) |>
# 각 경우에 따라 확인해야하는 상 하단의 가격이 존재하므로 관찰가격 upper,lower 변수 추가
    fmutate(range_lower_bound = fifelse( (탈출_first == 손절_first)&(손절_first ==buy_n_days),
                                         목표손절가 ,
                                         fifelse( (탈출_first == 손절_first)&(손절_first !=buy_n_days) ,
                                                  목표손절가,
                                                  fifelse( (탈출_first != buy_n_days)&(손절_first ==buy_n_days),
                                                           목표손절가     ,
                                                           목표구매가   )
                                         ) )) |> 
    fmutate(range_upper_bound = fifelse( (탈출_first == 손절_first)&(손절_first ==buy_n_days),
                                         목표익절가 ,
                                         fifelse( (탈출_first == 손절_first)&(손절_first !=buy_n_days) ,
                                                  목표익절가,
                                                  fifelse( (탈출_first != buy_n_days)&(손절_first ==buy_n_days),
                                                           목표구매가     ,
                                                           목표익절가   )
                                         ) )) %>% 
    
    
    fselect(시장구분,check_case,종목명,종목코드,목표구매가,buy_n_days,
                names(.) %>% grep(pattern = "탈출|손절|익절|sad|_date|range"))|>
    fmutate(거래일자 =str_remove_all(to_check_date,"-") %>% as.numeric, 
                double_check = 0 )->for_pull_data_from_db
  
  # 두번 체크해야 하는 경우.
  # 첫번째 탈출날짜와 구매가 같은날짜인데, 확인해보니 목표익절가가격->목표구매가 여서 그날 구매만 됐다고 하자.
  # 그래서 첫번째 탈출날짜는 무의미해졌는데 두번째 탈출날짜와 손절 날짜가 같을 수 있다. 이 경우에 대해 확인이 필요하다.
  # 즉 여기서는 같을 '수' 있는 case를 미리 뽑아 순서를 확인하고 진짜 확인해야 하는 경우가 있다면 여기서 계산해둔 값으로 갈아 끼우는 방식이다.  
  double_check %>% 
    filter( (탈출_first==buy_n_days&탈출_second==손절_first)&탈출_second!=10000  )  %>% 
    fmutate(to_check_date =  탈출_second_date,
            check_case = "탈=손!=구",
            range_lower_bound = 목표손절가, 
            range_upper_bound = 목표익절가 ) %>% 
    fselect(시장구분,check_case,종목명,종목코드,목표구매가,buy_n_days,
                names(.) %>% grep(pattern = "탈출|손절|익절|sad|_date|range"))|>
    fmutate(거래일자 =str_remove_all(to_check_date,"-") %>% as.numeric,double_check = 1)->double_check_tbl
  
  for_pull_data_from_db<- bind_rows(for_pull_data_from_db,double_check_tbl)
  # for_pull_data_from_db와 bind_row해서 한번에 데이터 가져오기
  gc()
  print("일중데이터db에 연결합니다.")
  tic()
  
  con <- DBI::dbConnect(RSQLite::SQLite(),
                        dbname="C:/_sqlite/krx_2019_2022.db" ) # db파일의 경로로 수정
  
  for_detail<- inner_join(x = copy_to(dest = con, df = for_pull_data_from_db,overwrite = TRUE ),
                          y = tbl(con,"iljung"),
                          by = c("거래일자","종목코드")) %>% collect()
  print("일중데이터db에서 데이터 가져오는데 소요된 시간:")
  toc()
  
  dbDisconnect(con)
  gc()
  
  # for_detail %>% names()
  for_detail %>% 
    fmutate(across(.cols = names(.) %>% grep(pattern = "_date"),.fns = as_date)) |> 
    fgroup_by(to_check_date,check_case,종목코드,filtered_date) |>
    fmutate(
      buy         = fifelse(목표구매가>=저가, 1, 0,na = 0),
      touch_lower = fifelse(range_lower_bound>=저가,1,0,na = 0),
      touch_upper = fifelse(range_upper_bound<=고가,1,0,na = 0),
      # touch가 1이면 상단가격 먼저 터치, 0이면 하단 터치 
      touch =fifelse(check_case!="탈=손=구",
                     fifelse(which.max(touch_lower)<=which.max(touch_upper)  , 0  ,  1),
                     
                     
                     # 탈=손=구에대해서, 구매이후 인덱스가 둘다 하나씩이상 있으면, 
                     ifelse(length(which(touch_lower[-(1:which.max(buy))]==1))>=1 &length(which(touch_upper[-(1:which.max(buy))]==1))>=1,  
                            #두개인덱스 중에서 빠른거로 반환하고
                            fifelse(which.max(touch_lower[-(1:which.max(buy))])<which.max(touch_upper[-(1:which.max(buy))])  , 0  ,  1),
                            # 둘중 하나라도 없는데 , 그중 lower bound인덱스가 없으면
                            fifelse(length(which(touch_lower[-(1:which.max(buy))]==1))==0 ,
                                    # 상단에 인덱스가 있을거고 upperbound 인덱스가 없으면 lowerbound  인덱스가 있을것이다.
                                    0,1) # 구매 이후 손절이든 탈출이든 있을것.
                     )
      ),
      touch = fifelse(check_case =="탈=구!=손" & touch==1 & length(which(touch_upper[-(1:which.max(touch_upper))]==1) ) >=1,
                      0,# 탈출 ->구매->탈출
                      touch),
      
      탈출_first =fifelse((check_case=="탈=손=구") &(touch == 1),10000 ,탈출_first),
      탈출_first =fifelse((check_case=="탈=구!=손") &(touch == 1),탈출_second,탈출_first),
      탈출_first =fifelse((check_case=="탈=손!=구") &(touch == 0),10000,탈출_first), # case 덧씌우기
      실제익절가_first =fifelse((check_case=="탈=구!=손") &(touch == 1),실제익절가_second,실제익절가_first),
            
      
      손절_first =fifelse((check_case=="탈=손=구") &(touch == 0),10000,손절_first),    
      손절_first =fifelse((check_case=="탈=손!=구") &(touch == 1),10000,손절_first),    
      
      # 실제구매가가 목표손절가이하면 0, 나중에 안사는걸로 filtering하기 위해 0으로 지정.
      # 이 경우에대해 하방 최대 폭에 대한 조건이 없어지기 때문이다.
      # 이부분에 대해서 자동매매서비스 도입시 매일 장 시작전에 예상체결가 데이터가 필요할 것으로 보인다.
      손절_first =fifelse((check_case=="탈!=구=손" )&(touch == 0),0,손절_first),
      sad_count = fifelse((check_case!="탈!=구=손")&(touch==1) , sad_count_탈second, sad_count) 
      
      
      
    ) |> 
    fsummarise(
      touch= touch[1],
      buy_n_days=buy_n_days[1],
      탈출_first = 탈출_first[1] ,
      손절_first = 손절_first[1] ,
      탈출_second = 탈출_second[1],
      실제익절가_first= 실제익절가_first[1],
      sad_count= sad_count[1],
      double_check = double_check[1]
      
    ) |>
    fungroup()->check_order # 순서확인후 결과 저장한 변수 : check_order
  gc()
  
  check_order |> 
    filter(double_check==1) |>
    select(1:5,7,8)->double_check_list
  
  
  keys <- c("filtered_date", "종목코드")
  
    # 두번체크해야하는 경우가 있었다면, 두번 체크한 결과로 대체
  check_order |> 
    filter(double_check==0) |>
    semi_join(double_check_list, keys) |>
    select(-matches(setdiff(names(double_check_list), keys))) |>
    left_join(double_check_list, keys) |>
    bind_rows(check_order %>% 
                filter(double_check==0)%>% anti_join(double_check_list, keys)) ->check_order
  
  # 기존 results파일에 순서확인한 결과값으로 채워넣기.
  adjusted_results<- results |>
    semi_join(check_order, keys) |>
    select(-matches(setdiff(names(check_order), keys))) |>
    left_join(check_order, keys) |>
    bind_rows(results %>% anti_join(check_order, keys)) |>
    # 구매할때 목표손절가로 내려와있으면 구매 x
    mutate(구매 = fifelse(손절_first==0 , 0, 구매),
             탈출_first = fifelse(탈출_first != 10000 & 손절_first != 10000 & 탈출_first > 손절_first, 10000, 탈출_first),
             손절_first = fifelse(탈출_first != 10000 & 손절_first != 10000 & 손절_first > 탈출_first, 10000, 손절_first)
             
    ) |> 
    select(-matches("second"),-matches("first_date"),-touch,-matches("check"),-matches("목표"))
  
  gc()
  return(adjusted_results) #최종결과 반환.
}

# back_test 결과 tbl에서 최종 결과 산출 함수 
result_func <- function(data){
  
  buy_num <- data |>filter(구매==1) |>nrow()
  #no_buy_num<-data |>filter(구매==0) |>nrow()
  escape_num <- data |> filter(구매==1 & 탈출_first!=10000) |>nrow()
  stop_num <- data |>filter(구매==1 &손절_first!=10000) |>nrow()
  
  
  #ratio_buy <- buy_num/(buy_num+no_buy_num)
  ratio_escape <- escape_num/buy_num
  ratio_stop_loss <- stop_num/buy_num
  
  ratio_date <-  data |>
    fmutate(min_탈출_손절 = fifelse(탈출_first>손절_first, 손절_first,탈출_first)) |>
    fmutate(sad_ratio = fifelse((탈출_first==10000) &(손절_first==10000) , sad_count/total_trade_days,
                                sad_count/min_탈출_손절)) |>
    fmutate(ending_position = fifelse(탈출_first==10000 &손절_first==10000, "최종가매도",
                                        if_else(탈출_first !=10000,"탈출","손절")) |>as_factor()) |>
    fmutate(ending_return = fifelse(ending_position =="최종가매도",
                                    final_price/실제구매가-1 ,
                                    if_else(ending_position=="탈출",
                                            실제익절가_first/실제구매가-1,
                                            실제손절가_first/실제구매가-1))) %>% {.->>for_buy_ratio}|>
    filter(구매==1) |> # 3일내 구매성공한 종목들에 대해서,
    fgroup_by(filtered_date,시장구분) |>
    fsummarise(
      슬픈_비율 = mean(sad_ratio,na.rm=T),
      탈출_비율 = sum(탈출_first!=10000)/length(종목코드),
      손절_비율 = sum(손절_first!=10000)/length(종목코드),
      최대수익률_평균 = mean(max_price/실제구매가)-1,
      최소수익률_평균 = mean(min_price/실제구매가)-1 , 
      기대수익률 = mean(ending_return),
      total_n_ticker = length(종목코드)
      
    ) |>
    fungroup()
  
  for_buy_ratio |>
    fgroup_by(filtered_date,시장구분) |>
    fsummarise(
      
      구매성공률 = sum(구매,na.rm = T)/length(종목코드)
    ) |>
    fungroup()->buy_ratio
  
  ratio_date<- left_join(ratio_date,buy_ratio)
  
  obj <- list(ratio_escape = ratio_escape,
              ratio_stop_loss = ratio_stop_loss,
              ratio_date = ratio_date)
  return(obj)
}




# 제 5장. 구현

### 5-1. 최종 필터링 (back_test에 넣는 데이터 생성)


임의로 거시변수와 종목 필터링 조건을 지정해주어 back_test 함수에 적용하였습니다. 임의로 지정한 조건들은 다음과 같습니다.  
  
| 변수   | 조건    |  
|:-------:| :------: |
| 전략검증기간  | 2019-09-01 ~ 2020-12-23  | 
| 콜금리 | - |
|회사채수익률| 설정안함 |
|원/달러 환율| 설정안함 |  
| 소비자물가지수 | 설정안함 | 
|국제유가| 설정안함 |
|뉴스심리지수 | 설정안함 | 
|원/위안 환율| 설정안함 |
|금| 설정안함|
|역PER | 상위 20%~80% | 
|시가총액 | 상위 20%~80% |
|유동성 | 상위 10~50%|
|거래빈도 기준 거래속도(30거래일) | 상위 10%~80% |
|20거래일고가대비등락률|상위 10%~70%|
|일중수익률변동성|상위 10%~70%|
|목표수익률 |10%|
|손절가 | -5% |
|심리적하한선 | -1% |
|관찰기간 | 30일 |
|구매결정가격 |-1% |
|ESG등급| 설정안함 |



In [ ]:

processed_data |> 
  # 필터링 위해서는 , 최소한 거래량이 있어야하고, 관리종목대상으로는 투자안하기로 (투자자 보호)
  filter(거래량>0) |>  
  # 필터링 되는날 기준으로는 제거 가능하지만, 필터링되고 30일간 관찰하는 과정에서 관리종목으로 바뀌는 종목들은 투자 될 수 있음.
  filter(!(소속부 %in% c("관리종목(소속부없음)","투자주의환기종목(소속부없음)","SPAC(소속부없음)"))) |> 
  #전체기간에 대해 계산하기 위한 변수(total_date)
  filter(open_date >="2019-07-01" &open_date<= "2022-06-30") %>%  
  {.->> total_date} %>%  
  #____확인하고 싶은 날짜 필터링----------------------
  filter(open_date >="2019-09-01" &open_date<= "2020-12-23") %>%  
  # 설정기간내 전체종목= selected_date
  {.->> selected_date} %>% 
  # ECOS 데이터 필터링
  # ____안넣을꺼면 주석처리.-----------------------------------------
#filter(`콜금리(익일물)(연%) _momentum`>0) %>%
#filter(`회사채수익률(3년,AA-)(연%) _momentum`>0) %>%
#filter(`원/달러 환율(종가)(원) _momentum`>0) %>%
#filter(`소비자물가지수(2020=100) _momentum`>0) %>%
#filter(`국제유가(Dubai, 현물)(U$/bbl) _momentum`<0) %>%
#filter(`뉴스심리지수() _momentum`>0) %>%

# 추가 변수  
#filter(`원/위안 환율(종가)(원) _momentum`>0) %>%
#filter(`금($/OZ) _momentum`>0) %>%


# ESG 데이터 필터링 

# ____안넣고 싶은게 있으면 지우면 됨.------------------------------
filter(ESG등급 %in% c("A+","A","B+","B","C","D","미지정")) |>
  
  # 거래 정보(default 상위 30%)
  
  filter(new_PER_percentile  %>% between( 20,80 )) |>
  filter(시가총액_percentile %>% between( 20,80)) |>
  filter(유동성_percentile   %>% between( 10,50 )) |>
  filter(`거래빈도 기준 거래속도(30거래일)_percentile`%>% between( 10, 80)) |>
  # 프리미엄변수 ex
  filter(`20거래일고가대비등락률_percentile`%>% between( 10,70 )) |> 
  filter(일중수익률변동성_percentile        %>% between( 10,70 ) )  -> filtered_trade_data
  # fgroup_by(종목코드) |> 
  # fmutate(selected_period_momentum = fifelse(종가[length(종가)]-종가[1]>=0,"+","-") ) |> 
  # fungroup() |> 
  #filter(selected_period_momentum %in% c("+","-"))  
#마지막 4줄의 코드는, 추후에 업데이트 할 수도 있는 부분인데,
#필터링 된 후, 2차적으로 필터링 된것들에서 추가 조건을 거는 것이다. 위의 예시는 필터링된 것들 중, 설정기간의 처음 과 끝만 비교했을때
# 상승 추세인 것들만 필터링 할 수 있는 코드이다. 

#rm(list=ls() %>% grep(pattern ="filtered_trade_data",value = T,invert = T) )
filtered_trade_data


open_date,종목명,종목코드,시장구분,소속부,종가,거래량,ESG등급,콜금리(익일물)(연%) _momentum,"회사채수익률(3년,AA-)(연%) _momentum",⋯,"국제유가(Dubai, 현물)(U$/bbl) _momentum",금($/OZ) _momentum,뉴스심리지수() _momentum,거래량_lag,시가총액_percentile,new_PER_percentile,유동성_percentile,거래빈도 기준 거래속도(30거래일)_percentile,20거래일고가대비등락률_percentile,일중수익률변동성_percentile
<date>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<fct>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2019-09-02,AJ렌터카,068400,KOSPI,NA,10850,77858,미지정,-1,1,⋯,-1,1,1,16341,26.87916,59.41960,43.81541,26.07041,31.87441,16.27022
2019-09-02,CNH,023460,KOSDAQ,중견기업부,1450,246600,미지정,-1,1,⋯,-1,1,1,584224,79.01998,60.32350,27.59277,68.83920,64.60514,18.17317
2019-09-02,DSC인베스트먼트,241520,KOSDAQ,벤처기업부,3115,73171,미지정,-1,1,⋯,-1,1,1,9526,77.73549,22.21694,39.86679,15.84206,61.60799,35.82303
2019-09-02,HB테크놀러지,078150,KOSDAQ,우량기업부,2905,1009391,미지정,-1,1,⋯,-1,1,1,1618424,27.87821,51.90295,17.31684,54.37678,47.19315,54.09134
2019-09-02,IHQ,003560,KOSPI,NA,1785,552608,미지정,-1,1,⋯,-1,1,1,48636,25.02379,66.12750,41.53187,19.83825,63.46337,59.70504
2019-09-02,KC코트렐,119650,KOSPI,NA,6490,104049,C,-1,1,⋯,-1,1,1,33844,61.32255,41.57945,24.31018,29.49572,37.86870,41.24643
2019-09-02,KG ETS,151860,KOSDAQ,우량기업부,3200,111503,B,-1,1,⋯,-1,1,1,478863,48.19220,21.74120,47.14558,72.31208,27.35490,33.15890
2019-09-02,KH바텍,060720,KOSDAQ,중견기업부,9050,102602,B,-1,1,⋯,-1,1,1,16020,32.11227,66.12750,34.63368,41.10371,54.90010,44.52902
2019-09-02,KNN,058400,KOSDAQ,우량기업부,1150,2690569,미지정,-1,1,⋯,-1,1,1,1500409,38.05899,40.91342,11.51284,45.29020,51.42721,17.79258


In [ ]:

# default 값
chance = 0.1
minus = -0.05
obs_period = 30
a = -0.01
sad = a
# 설정기간의 마지막을 end_date로 저장
end_date<- max(selected_date$open_date)


In [ ]:

# 이걸 이제 back_test함수에 넣고 , 반환되는 값들을 통해 시각화 하면 됨.

tictoc::tic()

result_back_test<- total_date |>
  # 미래참조편향을 고려하여 설정기간 마지막날 기준으로 탈출률이 계산되지 않는 항목 삭제
  filter(open_date <=max(total_date$open_date)-obs_period) %>%
  back_test_final(data       = .   ,
                  chance     = chance,
                  minus      = minus ,
                  sad        = sad,
                  obs_period = obs_period,
                  a          = a)

tictoc::toc()

#로컬에서 돌리면 일별데이터 1차분석 시간 + 일중데이터 db에서 데이터 가져오는시간에 조금의 시간만 더하면 전체 계산시간인데,
#주피터 노트북상에서 돌리니 시간이 더 걸림. 

#chance = 0.1
#minus = -0.05
#obs_period = 30
#a = -0.01
#sad = a 조건들에 대해서 전체기간, 전체종목에 대해서 10분정도 걸림)

result_back_test

[1] "분석을 시작합니다."
[1] "일별데이터를 통해 1차 분석이 완료되었습니다."
[1] "소요시간:"
484.98 sec elapsed
[1] "거래순서 확인이 필요한 경우에 대해서 일중데이터를 통해 확인을 시작합니다."
[1] "일중데이터db에 연결합니다."
[1] "일중데이터db에서 데이터 가져오는데 소요된 시간:"
81.05 sec elapsed
1119.11 sec elapsed


filtered_date,종목코드,시장구분,종목명,실제구매가,실제손절가_first,min_price,max_price,total_trade_days,final_price,final_volume,filtereddate_종가,obs_expired,구매,buy_n_days,실제익절가_first,sad_count,탈출_first,손절_first
<date>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2019-07-01,005290,KOSDAQ,동진쎄미켐,11700.000,11100.000,11500.000,18800.000,22,13650.000,3856377.00,11850.000,2019-07-31,1,1,12900.00,1,1,10000
2019-07-01,005390,KOSPI,신성통상,1085.000,1030.000,1080.000,1560.000,22,1385.000,21507301.00,1100.000,2019-07-31,1,2,1195.00,0,2,10000
2019-07-01,006060,KOSPI,화승인더,8990.000,8560.000,8140.000,11850.000,22,8500.000,156823.00,9120.000,2019-07-31,1,1,9930.00,1,1,10000
2019-07-01,006880,KOSPI,신송홀딩스,5280.000,5060.000,5160.000,6740.000,22,5340.000,322024.00,5390.000,2019-07-31,1,1,5870.00,0,1,10000
2019-07-01,007860,KOSPI,서연,4945.000,4640.000,3305.000,5490.000,22,3410.000,89924.00,4995.000,2019-07-31,1,1,5440.00,1,1,10000
2019-07-01,009520,KOSDAQ,포스코엠텍,6510.000,6180.000,4585.000,7350.000,22,4935.000,298908.00,6580.000,2019-07-31,1,1,7170.00,1,1,10000
2019-07-01,009620,KOSDAQ,삼보산업,2291.505,2173.527,1497.419,2722.581,22,1570.022,1703737.96,2364.108,2019-07-31,1,1,2575.00,1,1,10000
2019-07-01,010040,KOSPI,한국내화,5790.000,5500.000,4110.000,6380.000,22,4470.000,180066.00,5850.000,2019-07-31,1,1,6370.00,1,1,10000
2019-07-01,011200,KOSPI,현대상선,3805.000,3610.000,3195.000,4280.000,22,3305.000,636784.00,3845.000,2019-07-31,1,1,4190.00,1,1,10000


In [ ]:
# 위의 결과값을 미리 저장해둔 rds파일을 불러와서 진행

result_back_test<- readRDS(file = "C:/Users/shj/Documents/2022_summer_project/krx_repo/back_test_chance=0.1,minus=-0.05,a=-0.01,obs_period=30.rds")
result_back_test


filtered_date,종목코드,시장구분,종목명,실제구매가,실제손절가_first,min_price,max_price,total_trade_days,final_price,final_volume,filtereddate_종가,obs_expired,구매,buy_n_days,실제익절가_first,sad_count,탈출_first,손절_first
<date>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2019-07-01,005290,KOSDAQ,동진쎄미켐,11700.000,11100.000,11500.000,18800.000,22,13650.000,3856377.00,11850.000,2019-07-31,1,1,12900.00,1,1,10000
2019-07-01,005390,KOSPI,신성통상,1085.000,1030.000,1080.000,1560.000,22,1385.000,21507301.00,1100.000,2019-07-31,1,2,1195.00,0,2,10000
2019-07-01,006060,KOSPI,화승인더,8990.000,8560.000,8140.000,11850.000,22,8500.000,156823.00,9120.000,2019-07-31,1,1,9930.00,1,1,10000
2019-07-01,006880,KOSPI,신송홀딩스,5280.000,5060.000,5160.000,6740.000,22,5340.000,322024.00,5390.000,2019-07-31,1,1,5870.00,0,1,10000
2019-07-01,007860,KOSPI,서연,4945.000,4640.000,3305.000,5490.000,22,3410.000,89924.00,4995.000,2019-07-31,1,1,5440.00,1,1,10000
2019-07-01,009520,KOSDAQ,포스코엠텍,6510.000,6180.000,4585.000,7350.000,22,4935.000,298908.00,6580.000,2019-07-31,1,1,7170.00,1,1,10000
2019-07-01,009620,KOSDAQ,삼보산업,2291.505,2173.527,1497.419,2722.581,22,1570.022,1703737.96,2364.108,2019-07-31,1,1,2575.00,1,1,10000
2019-07-01,010040,KOSPI,한국내화,5790.000,5500.000,4110.000,6380.000,22,4470.000,180066.00,5850.000,2019-07-31,1,1,6370.00,1,1,10000
2019-07-01,011200,KOSPI,현대상선,3805.000,3610.000,3195.000,4280.000,22,3305.000,636784.00,3845.000,2019-07-31,1,1,4190.00,1,1,10000


back_test 함수의 output으로 구현한 값들에 대한 설명입니다.

*   탈출률 (ratio_escape) : 탈출한 종목수를 구매한 종목수로 나누어 계산하며, 날짜별로 생성됩니다. 관찰 일수 내 특정 날짜의 고가가 목표 수익률에 해당하는 가격보다 높으면 탈출로 간주합니다. 
*   손절률 (ratio_stop_loss) : 손절한 종목수를 구매한 종목수로 나누어 계산하며, 날짜별로 생성됩니다. 관찰 일수 내 특정 날짜의 저가가 최대로 허용한 손실 가격보다 낮으면 손절로 간주합니다. 
*   탈출과 손절이 동시에 일어난 날짜 (ratio_stop_loss_eacape_date) : 매분 매초 변동하는 가격을 데이터로 알 수 없기 때문에, 탈출/손절 여부를 고가와 저가로 판단하게 됩니다. 하루 주가의 변동폭이 커서 탈출 임계치와 손절 임계치를 하루에 모두 도달해버리는 경우가 발생할 수 있는데, 이 경우는 추후 일중 데이터로 먼저 도달한 임계치를 확인하여 보완할 예정입니다. 일중 데이터가 부족해 현재는 보완하지 못하였으며, back_test 내에서 동시에 발생하는 경우에는 탈출로 반환하도록 하였습니다. ==> 일별로 관찰하고자하는 가격의 하단,상단을 정해놓고 일중 데이터를 통해 순서를 확인하였습니다.


*   날짜별 손절 및 탈출 비율 (escape_stop_day) : 앞서 계산한 탈출률과 손절률은 전체 기간의 종목수를 기반으로 계산한 한편, 개별 날짜별 종목수를 기반으로도 탈출률과 손절률을 계산할 수 있습니다.

### 5-2. 벤치마크 선정

**선택한 날짜에 대해 전체 종목에 대해 동일하게 매수 매도전략 실시**

전략의 유의성을 검정하기 위해 설정기간 전체의 매일매일 전 종목에 대해 

필터링된날 종가*(1+a) = 목표구매가

매수(목표구매가) , 매도- 목표구매가(1+chance),목표구매가*(1+minus) 를 정해놓고 예약주문 체결을 가정했을시 결과와

거시변수 조합, 종목필터링 조건으로 나온 날짜와 해당하는 종목들에 매수 매도계획을 적용했을때의 결과를 비교


In [ ]:
# 설정 기간으로 필터링 된 날 종목전략을 통해 반환된 종목들에 대한 back_test 결과값
filtered_result_back_test<- inner_join(filtered_trade_data %>% 
                                         filter(open_date<=end_date-obs_period) %>% 
                                         rename(filtered_date = open_date) %>% 
                                         select(filtered_date, 종목코드),
                                       result_back_test,
                                       by =c("filtered_date","종목코드") )

# 설정 기간으로 필터링 된 날 전체 종목들에 대한 back_test 결과값 (벤치마크)
contrast_back_test<- inner_join(selected_date %>% 
                                         filter(open_date<= end_date-obs_period) %>%  
                                         rename(filtered_date = open_date) %>% 
                                         select(filtered_date, 종목코드),
                                       result_back_test,
                                       by =c("filtered_date","종목코드") )

# 전체기간, 전체종목에 대해 back_test결과값을 요약해주는 함수 적용하여 값 저장
temp<-result_func(result_back_test)
total_escape<-temp %>% pluck(1) # 탈출 비율
total_stop_loss <-temp %>% pluck(2) # 손절 비율
total_back_test_result <-temp %>% pluck(3)  # 날짜별 손절 및 탈출 비율등등 반환
total_back_test_result %>% head()


#  설정 기간으로 중 거시변수 & 종목전략을 통해 반환된 날짜별 종목들의 back_test 결과값을 요약해주는 함수 적용하여 값 저장 
temp<-result_func(filtered_result_back_test)

strategy_escape <- temp %>% pluck(1)# 탈출 비율
strategy_stop_loss <- temp %>% pluck(2)# 손절 비율
strategy_back_test_result <- temp %>% pluck(3)# 날짜별 손절 및 탈출 비율등등 반환
strategy_back_test_result %>% head()

#  설정 기간으로 중 전체종목에 대해 back_test결과값을 요약해주는 함수 적용하여 값 저장

temp<-result_func(contrast_back_test)

contrast_strategy_escape <- temp %>% pluck(1)# 탈출 비율
contrast_strategy_stop_loss <- temp %>% pluck(2)# 손절 비율
contrast_strategy_back_test_result <- temp %>% pluck(3)# 날짜별 손절 및 탈출 비율등등 반환

contrast_strategy_back_test_result %>% head()


Joining, by = c("filtered_date", "시장구분")


filtered_date,시장구분,슬픈_비율,탈출_비율,손절_비율,최대수익률_평균,최소수익률_평균,기대수익률,total_n_ticker,구매성공률
<date>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
2019-07-01,KOSDAQ,0.7257822,0.2539370,0.6771654,0.10350271,-0.1681517,-0.02414964,1016,0.8403639
2019-07-01,KOSPI,0.7409618,0.1500000,0.6000000,0.06433493,-0.1398740,-0.03843460,640,0.8173691
2019-07-02,KOSDAQ,0.8088872,0.2165242,0.7160494,0.09265841,-0.1738226,-0.03097276,1053,0.8709677
2019-07-02,KOSPI,0.7926178,0.1200608,0.6458967,0.05795891,-0.1413487,-0.04451144,658,0.8403576
2019-07-03,KOSDAQ,0.7872165,0.2299559,0.6863436,0.09549442,-0.1705072,-0.02828545,1135,0.9387924
2019-07-03,KOSPI,0.7669849,0.1348974,0.6422287,0.06277959,-0.1393932,-0.04266423,682,0.8687898


Joining, by = c("filtered_date", "시장구분")


filtered_date,시장구분,슬픈_비율,탈출_비율,손절_비율,최대수익률_평균,최소수익률_평균,기대수익률,total_n_ticker,구매성공률
<date>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
2019-09-02,KOSDAQ,0.3780805,0.5443038,0.20253165,0.1589826,-0.07377547,0.04163516,79,0.6269841
2019-09-02,KOSPI,0.2972885,0.6521739,0.08695652,0.1765651,-0.03671384,0.06342049,23,0.5750000
2019-09-03,KOSDAQ,0.3748125,0.5405405,0.20270270,0.1385297,-0.08543503,0.03961636,74,0.6218487
2019-09-03,KOSPI,0.2959942,0.5652174,0.17391304,0.1846205,-0.04448184,0.04881113,23,0.6969697
2019-09-04,KOSDAQ,0.5791446,0.2708333,0.33333333,0.1046578,-0.09453679,-0.01369409,96,0.7619048
2019-09-04,KOSPI,0.4115178,0.5357143,0.17857143,0.1687565,-0.06570793,0.03863142,28,0.6829268


Joining, by = c("filtered_date", "시장구분")


filtered_date,시장구분,슬픈_비율,탈출_비율,손절_비율,최대수익률_평균,최소수익률_평균,기대수익률,total_n_ticker,구매성공률
<date>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
2019-09-02,KOSDAQ,0.3794543,0.5265152,0.14898990,0.1618019,-0.06491634,0.04184459,792,0.6518519
2019-09-02,KOSPI,0.2928344,0.4210526,0.07368421,0.1224689,-0.03651300,0.03927640,475,0.6097561
2019-09-03,KOSDAQ,0.3824407,0.5100402,0.15261044,0.1555560,-0.06624062,0.04209753,747,0.6148148
2019-09-03,KOSPI,0.2965131,0.3941909,0.08713693,0.1215403,-0.03912619,0.03396106,482,0.6195373
2019-09-04,KOSDAQ,0.4981921,0.4087760,0.21709007,0.1354101,-0.07730073,0.02033304,866,0.7127572
2019-09-04,KOSPI,0.3665486,0.3115079,0.10317460,0.1057865,-0.04711735,0.01895949,504,0.6469833


### 5-3.  시각화

**시각화 결과물이 interactive한 것은 jupyter notebook 상에서 결과 저장이 안되는 것으로 확인하여 어쩔 수 없이 캡쳐화면으로 대체하였습니다.**

**1) 설정한 전략으로 필터링된 날짜들의 종목수**

In [ ]:

# 거시변수조합별 기대수익률 보기 --------------------------------------------------------

new_feature<- macro_filter %>% 
  filter(complete.cases(.)) %>% 
  rename(filtered_date = time) %>%
  mutate(filtered_date = ymd(filtered_date)) %>% 
  mutate(comb_ecos = paste0(fifelse(`콜금리(익일물)(연%) _momentum`==1,"콜","(콜)"), # +이면 콜, -이면 ()붙여서 반환
                            fifelse(`회사채수익률(3년,AA-)(연%) _momentum`==1,"회","(회)"),
                            fifelse(`원/달러 환율(종가)(원) _momentum` ==1,"달","(달)"),
                            fifelse(`원/위안 환율(종가)(원) _momentum`==1,"위","(위)"),
                            fifelse(`소비자물가지수(2020=100) _momentum`==1,"소","(소)"),
                            fifelse(`국제유가(Dubai, 현물)(U$/bbl) _momentum`==1,"유","(유)"),
                            fifelse(`뉴스심리지수() _momentum`==1,"뉴","(뉴)"),
                            fifelse(`금($/OZ) _momentum`==1,"금","(금)") ))


#KOSPI,KOSDAQ 지수 데이터 읽어오기

KOSPI_KOSDAQ <- read_csv("C:/Users/shj/Documents/2022_summer_project/krx_repo/KOSPI_KOSDAQ_2019_2022.csv")



KOSPI_KOSDAQ %<>%
  rename(filtered_date = time) %>%
  mutate(across(.cols = everything(), .fns = ~na.locf(.x,na.rm = FALSE) )) %>% 
# 필터링 된날 기준으로 기대수익률과 , 그때로부터 동일한 관찰기간의 시장 수익률을 비교하기 위한 변수 생성.
  mutate(across(.cols = -filtered_date, ~lead(.,n=obs_period) ,.names = "{.col}_lead")) %>% 
  mutate(KOSPI = `코스피지수(1980.01.04=100)_lead`/`코스피지수(1980.01.04=100)`-1, 
         KOSDAQ =`코스닥지수(1996.07.01=1000)_lead`/ `코스닥지수(1996.07.01=1000)`-1) %>% 
  select(filtered_date,KOSPI,KOSDAQ)

colnames(KOSPI_KOSDAQ) <- c("filtered_date",str_glue("KOSPI_{obs_period}days_return"),str_glue("KOSDAQ_{obs_period}days_return"))
 

left_join(left_join( strategy_back_test_result ,KOSPI_KOSDAQ),
          n_day_data |> 
            group_by(open_date,시장구분) |> 
            summarise(ticker_list = n()),by = c("filtered_date"="open_date","시장구분")) |>
  group_by(filtered_date,시장구분) |> 
  mutate(n_ticker_ratio =(total_n_ticker/ticker_list) ) |> 
  ungroup() ->imsi


temp_1<- left_join( strategy_back_test_result ,KOSPI_KOSDAQ)


temp<- left_join(temp_1,new_feature)

temp %>% head()
#macro 와 주가지수와의 상관관계를 통해 어떤 거시변수와 연관이 있는지 확인해보자.

# 내 전략의 시장대비 승률
temp %>%
# 수수료(0.25%) 까지 넣었을때 시장대비 승률
  mutate(win_market = fifelse(시장구분=="KOSPI",기대수익률>KOSPI_30days_return+0.0025 ,기대수익률>KOSDAQ_30days_return+0.0025 )) %>% 
  group_by(시장구분) %>%
  summarise(win_ratio = fsum(win_market)/n(),
            ndays = n(),
            tickers = fsum(total_n_ticker)
  )


# 내 전략 & ecos조합별로 봤을때 시장대비 승률 
temp %>%
  mutate(win_market = fifelse(시장구분=="KOSPI",기대수익률>=KOSPI_30days_return ,기대수익률>=KOSDAQ_30days_return )) %>% 
  group_by(시장구분,comb_ecos) %>%
  summarise(win_ratio = sum(win_market)/n(),
            ndays = n(),
            tickers = fsum(total_n_ticker)  # ndays>5이상인거만 본다던가 등등 .
            ) %>% 
  arrange(-win_ratio)






# 동일한 매수,매도 전략에 대해서 전체 종목으로 투자했을시 승률도 보여주고,
# 전체 종목에 투자한것과 종목필터링전략 적용한것과 비교한 승률도 보여줘야할듯.
  



temp %>% 
  mutate(filtered_date = str_remove_all(filtered_date,pattern = "-")) %>% 
  group_split(comb_ecos) ->list_filtered_date_by_comb

temp %>%
  group_by(comb_ecos) %>% 
  summarise(across(.cols = where(is.numeric),.fns = mean),
            n_days= n()) %>% 
  mutate(date_list = list_filtered_date_by_comb %>% 
           map(unique("filtered_date"))) %>% 
  arrange(-탈출_비율,-기대수익률) ->total_results

total_results %>% head()



Rows: 1277 Columns: 3
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
dbl  (2): 코스피지수(1980.01.04=100), 코스닥지수(1996.07.01=1000)
date (1): time

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Joining, by = "filtered_date"
`summarise()` has grouped output by 'open_date'. You can override using the `.groups` argument.
Joining, by = "filtered_date"
Joining, by = "filtered_date"


filtered_date,시장구분,슬픈_비율,탈출_비율,손절_비율,최대수익률_평균,최소수익률_평균,기대수익률,total_n_ticker,구매성공률,⋯,KOSDAQ_30days_return,콜금리(익일물)(연%) _momentum,"회사채수익률(3년,AA-)(연%) _momentum",원/달러 환율(종가)(원) _momentum,원/위안 환율(종가)(원) _momentum,소비자물가지수(2020=100) _momentum,"국제유가(Dubai, 현물)(U$/bbl) _momentum",금($/OZ) _momentum,뉴스심리지수() _momentum,comb_ecos
<date>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,⋯,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>
2019-09-02,KOSDAQ,0.3780805,0.5443038,0.20253165,0.1589826,-0.07377547,0.04163516,79,0.6269841,⋯,0.007582969,-1,1,-1,-1,1,-1,1,1,(콜)회(달)(위)소(유)뉴금
2019-09-02,KOSPI,0.2972885,0.6521739,0.08695652,0.1765651,-0.03671384,0.06342049,23,0.5750000,⋯,0.007582969,-1,1,-1,-1,1,-1,1,1,(콜)회(달)(위)소(유)뉴금
2019-09-03,KOSDAQ,0.3748125,0.5405405,0.20270270,0.1385297,-0.08543503,0.03961636,74,0.6218487,⋯,0.007241702,1,1,-1,-1,1,-1,1,1,콜회(달)(위)소(유)뉴금
2019-09-03,KOSPI,0.2959942,0.5652174,0.17391304,0.1846205,-0.04448184,0.04881113,23,0.6969697,⋯,0.007241702,1,1,-1,-1,1,-1,1,1,콜회(달)(위)소(유)뉴금
2019-09-04,KOSDAQ,0.5791446,0.2708333,0.33333333,0.1046578,-0.09453679,-0.01369409,96,0.7619048,⋯,-0.011870143,-1,-1,1,1,1,-1,1,-1,(콜)(회)달위소(유)(뉴)금
2019-09-04,KOSPI,0.4115178,0.5357143,0.17857143,0.1687565,-0.06570793,0.03863142,28,0.6829268,⋯,-0.011870143,-1,-1,1,1,1,-1,1,-1,(콜)(회)달위소(유)(뉴)금


시장구분,win_ratio,ndays,tickers
<chr>,<dbl>,<int>,<int>
KOSDAQ,0.4105960,302,37329
KOSPI,0.4238411,302,9139


`summarise()` has grouped output by '시장구분'. You can override using the `.groups` argument.


시장구분,comb_ecos,win_ratio,ndays,tickers
<chr>,<chr>,<dbl>,<int>,<int>
KOSDAQ,(콜)(회)(달)(위)(소)유(뉴)금,1,1,136
KOSDAQ,(콜)(회)(달)(위)소(유)뉴금,1,1,135
KOSDAQ,(콜)(회)(달)(위)소유(뉴)(금),1,3,417
KOSDAQ,(콜)(회)(달)(위)소유뉴(금),1,1,107
KOSDAQ,(콜)(회)(달)위소(유)(뉴)(금),1,1,73
KOSDAQ,(콜)(회)(달)위소유(뉴)(금),1,1,144
KOSDAQ,(콜)(회)달(위)소유(뉴)금,1,1,115
KOSDAQ,(콜)(회)달(위)소유뉴(금),1,2,238
KOSDAQ,(콜)(회)달(위)소유뉴금,1,1,152


comb_ecos,슬픈_비율,탈출_비율,손절_비율,최대수익률_평균,최소수익률_평균,기대수익률,total_n_ticker,구매성공률,KOSPI_30days_return,⋯,콜금리(익일물)(연%) _momentum,"회사채수익률(3년,AA-)(연%) _momentum",원/달러 환율(종가)(원) _momentum,원/위안 환율(종가)(원) _momentum,소비자물가지수(2020=100) _momentum,"국제유가(Dubai, 현물)(U$/bbl) _momentum",금($/OZ) _momentum,뉴스심리지수() _momentum,n_days,date_list
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<list>
콜회달(위)소(유)뉴(금),0.5252627,0.9965517,0.003448276,0.5542045,-0.02374813,0.13001179,94.5,0.94600281,0.09971299,⋯,1,1,1,-1,1,-1,-1,1,2,"20200327, 20200327"
콜(회)(달)(위)소(유)뉴(금),0.4290008,0.9707310,0.029268989,0.5832377,-0.03104905,0.09936102,68.5,0.82663415,0.10807973,⋯,1,-1,-1,-1,1,-1,-1,1,2,"20200325, 20200325"
콜회달위소(유)(뉴)(금),0.3856209,0.9705882,0.029411765,0.4361852,-0.03454023,0.09939278,10.0,0.10339147,0.18929545,⋯,1,1,1,1,1,-1,-1,-1,2,"20200324, 20200324"
(콜)회달(위)소(유)(뉴)(금),0.5356143,0.9617209,0.038279133,0.8809536,-0.03461869,0.10266008,59.0,0.63693399,0.31344502,⋯,-1,1,1,-1,1,-1,-1,-1,2,"20200319, 20200319"
(콜)(회)(달)(위)소(유)(뉴)(금),0.2916667,0.9444444,0.000000000,0.9387087,-0.01407355,0.09655780,5.5,0.06218328,0.27905643,⋯,-1,-1,-1,-1,1,-1,-1,-1,2,"20200323, 20200323"
(콜)(회)(달)(위)(소)(유)뉴(금),0.2940843,0.9092997,0.044776119,0.3697372,-0.03311697,0.09389906,40.0,0.36532671,0.12873238,⋯,-1,-1,-1,-1,-1,-1,-1,1,2,"20200403, 20200403"


**2) 설정한 전략(stratedy)vs 벤치마크(control) 탈출률, 손절률, 관찰 마지막날까지 매도하지 못한 종목의 비율 시각화**



In [ ]:

library(echarts4r)
library(plotly)
e_common(font_family = "helvetica", theme = "macarons")


comparison <- data.frame(type = rep(c("탈출률", "손절률", "기타"), each = 2), 
                         obj = rep(c("Control","Strategy"), times = 3),
                         val = c(contrast_strategy_escape, strategy_escape, 
                                 contrast_strategy_stop_loss, strategy_stop_loss, 
                                 1-contrast_strategy_escape-contrast_strategy_stop_loss, 1-strategy_escape-strategy_stop_loss))
comparison %<>%  mutate(val = round(val, 2),
                        type = factor(type, levels = c("탈출률", "손절률", "기타")))


#comparison |> group_by(type) |> 
#  e_charts(obj) |> 
#  e_bar(val, stack = "grp") |> 
#  e_tooltip(
#    trigger = "axis",
#    axisPointer = list(type = "shadow")
#  ) |> 
#  e_y_axis(
#    spiltArea = list(show = FALSE),
#    splitLine = list(show = FALSE),
#    show = FALSE
#  ) |> 
#  e_flip_coords()



<p align="center"><img src = "https://drive.google.com/uc?id=1R9hWpvgG0P932ToOhbby3JzXiOJs_OZn">

**3) 설정한 전략(stratedy)vs 벤치마크(control)의 레이더차트** 

In [ ]:

a <- strategy_back_test_result |> pull(탈출_비율) |> mean()
b <- 1 - strategy_back_test_result |> pull(손절_비율) |> mean()
c <- 1 - strategy_back_test_result |> pull(슬픈_비율) |> mean()
d <- strategy_back_test_result |> pull(기대수익률) |> mean()  # 일별 기대수익률의 단순평균을 하였습니다.
e <- strategy_back_test_result |> pull(탈출_비율) |> sd()

A <- contrast_strategy_back_test_result |> pull(탈출_비율) |> mean()
B <- 1 - contrast_strategy_back_test_result |> pull(손절_비율) |> mean()
C <- 1 - contrast_strategy_back_test_result |> pull(슬픈_비율) |> mean()
D <- contrast_strategy_back_test_result |> pull(기대수익률) |> mean()
E <- contrast_strategy_back_test_result |> pull(탈출_비율) |> sd()

df <- data.frame(
  type = c("탈출률", "1-손절률", "1-슬픈비율", "기대수익률", "탈출률_변동성"),
  val_1 = c(a,b,c,d,e) |> round(3),
  val_2 = c(A,B,C,D,E) |> round(3)
)

radar_max <- c(df$val_1, df$val_2) |> max() * 1.1

#df %<>%  mutate( big = ifelse(val_1 > val_2, val_1, val_2) * 1.2)
#df |> 
#  e_charts(type) |> 
#  e_radar(val_1, name = "나의 전략", 
#          areaStyle = TRUE, 
#          max = list(df$big)) |> 
#  e_radar(val_2, name = "Control") |>
#  e_radar_opts(radius = "80%",
#               axisName = list(fontSize = 15,
#                               fontWeight = "bold",
#                               color = "black")) |> 
#  e_legend(right = 10, bottom = "50%",
#           orient = "vertical") |> 
#  e_tooltip(trigger = "item") 


<p align="center"><img src = "https://drive.google.com/uc?id=1MdCf6MosB7Zc1_EyRd2yB3OdLOQY_NMX">

**4) 설정한 전략(stratedy)vs 시장지수(control) 수익률 비교** 

In [ ]:


IMSI <- imsi |> mutate(control = ifelse(시장구분 == "KOSPI", KOSPI_30days_return, KOSDAQ_30days_return)) |> 
  rename(나의전략 = 기대수익률) |> 
  select(filtered_date, 시장구분, 나의전략, control)


imsi |> 
  mutate(win_market = fifelse(시장구분=="KOSPI",기대수익률>KOSPI_30days_return+0.0025 ,기대수익률>KOSDAQ_30days_return+0.0025 )) |> 
  group_by(시장구분) %>%
  summarise(win_ratio = fsum(win_market)/n(),
            ndays = n(),
            tickers = fsum(total_n_ticker),  
            mean_n_ticker_ratio = mean(n_ticker_ratio), # 전체종목 대비 필터링된 종목비율
            mean_기대수익률= mean(기대수익률),
            mean_KOSPI  = mean(KOSPI_30days_return), 
            mean_KOSDAQ = mean(KOSDAQ_30days_return)
  )->result_table

result_table

y_min <- c(IMSI$나의전략, IMSI$control) |> min()
y_max <- c(IMSI$나의전략, IMSI$control) |> max()

e_common(font_family = "helvetica", theme = "macarons")

#IMSI |> group_by(시장구분) |> 
#  e_charts(filtered_date, timeline = TRUE) |> 
#  e_area(나의전략, smooth = TRUE, name = "나의 전략") |> 
#  e_area(control, y.index = 1, smooth = TRUE, name = "Control") |>  
#  e_y_axis(min = y_min, max = y_max,
#           formatter = e_axis_formatter(style = "percent")) |>
#  e_legend(right=0, bottom = "50%",
#           orient = "vertical") |>
#  e_title("KOSPI/KOSDAQ 시장대비 기대수익률 그래프") |> 
#  e_tooltip(trigger = "axis")





시장구분,win_ratio,ndays,tickers,mean_n_ticker_ratio,mean_기대수익률,mean_KOSPI,mean_KOSDAQ
<chr>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
KOSDAQ,0.4105960,302,37329,0.09123185,0.02787080,0.02298279,0.02910637
KOSPI,0.4238411,302,9139,0.03804294,0.02283704,0.02298279,0.02910637


<p align="center"><img src = "https://drive.google.com/uc?id=11eWq_RZ8ZUBqupup7yA8a2KtAHKw1o5y">

**5) 선택된 날짜의 필터링 된 종목 중 임의로 선택한 종목의 최근 추이를 보기 위한 candle 차트**

In [ ]:

candle <-n_day_data |> 
  select(open_date, 종목명, 시가, 종가, 저가, 고가, 거래량) |> 
  arrange(open_date) |> 
  filter(open_date >= "2019-07-01") |> 
  rename(filtered_date = open_date) |> 
  left_join(KOSPI_KOSDAQ) 
  



#candle |> 
#  filter(종목명 == "삼성전자") |> 
#  mutate(filtered_date = as.character(filtered_date)) |>
#  e_charts(filtered_date) |> 
#  e_candle(시가, 종가, 저가, 고가, name = "삼성전자") |> 
#  e_legend(show = FALSE) |> 
#  e_tooltip(
#    trigger = "axis",
#    axisPointer = list(type = "cross")
#  ) |> 
#  e_datazoom(x_index = 0, type = "inside",
#             startValue = as.character(min(selected_date$open_date)),
#             endValue = as.character(max(selected_date$open_date))) |>  
#  e_datazoom(y_index = 0, type = "inside")




Joining, by = "filtered_date"


<p align="center"><img src = "https://drive.google.com/uc?id=1GHg_2tPPhaOT4rI-TIRV9V6fsj258266">

**6) 설정 기간 내 거시변수 조합별 기대수익률**

In [ ]:

#total_results |> mutate(부호 = ifelse(기대수익률 > 0, "+", "-")) |> 
#  filter(n_days > 1) |> mutate(기대수익률 = round(기대수익률, 3)) |> 
#  arrange(기대수익률) |> 
#  group_by(부호) |> 
#  e_charts(comb_ecos) |> 
#  e_bar(기대수익률) |> 
#  e_flip_coords() |> 
#  e_legend(show = FALSE) |> 
#  e_x_axis(
#    formatter = e_axis_formatter(style = "percent"),
#    name = "기대수익률"
#  ) |> 
#  e_tooltip(
#    trigger = "item",
#    axisPointer = list(type = "shadow")
#    
#  ) |> 
#  e_y_axis(show = FALSE) |> 
#  e_title("거시변수 조합별 기대수익률",
#          "클릭 시 조합 확인 할 수 있음.") 





<p align="center"><img src = "https://drive.google.com/uc?id=1VBmbpboeCV9wlkUk7rRjouzyJLjqbQt9">

**7) 설정기간 내 거시변수 조합별 최소/최대 수익률**

In [ ]:

ttt <- total_results |> arrange(-최대수익률_평균) |> dplyr::slice(1:15)

p <- ttt |> 
  arrange(최대수익률_평균) |> 
  mutate(comb_ecos = factor(comb_ecos, levels = comb_ecos),
         최소수익률_평균 = round(최소수익률_평균, 3),
         최대수익률_평균 = round(최대수익률_평균, 3)) |> 
  ggplot() +
  geom_segment(aes(x = comb_ecos,xend = comb_ecos, y = 최소수익률_평균, yend = 최대수익률_평균),color = "#aeb6bf", size = 2, alpha = 0.7) +
  geom_point(aes(x = comb_ecos, y = 최대수익률_평균),
             color = "#02e0d9", size = 3) +
  geom_point(aes(x = comb_ecos, y = 최소수익률_평균), color = "#ff295e", size = 3) +
  coord_flip() +
  theme(axis.text.y = element_blank(),
        axis.ticks.y = element_blank(),
        panel.border = element_blank()) + 
  scale_y_continuous(name = "최소/최대 수익률",
                     labels = scales::label_percent()) +
  xlab("Combination of ECOS") +
  ggtitle("거시변수 조합별 최소/최대 수익률")

#ggplotly(p)



<p align="center"><img src = "https://drive.google.com/uc?id=1m480cvRG6f-nkKihc6ArS9SEJrihNSqv">

# 제 6장. 비즈니스 모델




### 6-1.비즈니스 모델 캔버스 
<p align="center"><img src = "https://drive.google.com/uc?id=1_qYeI12olhRW4CsRX8W3G8ssCzHkylX6">


### 6.2. Quantrol 만의 차별성
<p align="center"><img src = "https://drive.google.com/uc?id=1DQB9AxBQoH33bB__7yZOjfypqL8bRIO5">

### 6.3. Quantrol 의 기대효과
<p align="center"><img src = "https://drive.google.com/uc?id=1hbMSXHCDvJi-X7KLTcdbEIQd27ZEvdJy">

# 제 7장. 향후 보완할 점

### 7-1. 종목 전략 Grid Search

백테스팅 툴을 사용할 때 사용자가 일일이 종목조건을 조절해 가면서 좋은 전략을 찾기 위해서는 많은 시간이 걸립니다. 또한, 전략 조합의 개수가 너무 많기 때문에 좋은 전략을 찾아도 최적의 전략인지는 확인 할 수 없습니다. 따라서 모든 전략 조합을 탐색할 수 있는 기능을 만들어 제공하고자 합니다. 이러한 기능을 사용자에게 이용료를 받고 제공할 수도 있습니다. 아래 적은 grid search 코드는 그 기능을 일부 구현하여 향후 서비스 제공을 위한 가능성을 제시합니다.

**매수-매도 조건과 거시변수 조합 조건에 따른 최적의 종목조합을 찾는 그리드 서치 함수**


1. 매수-매도 조건과 거시변수 조합 조건에 원하는 조건을 입력합니다
2. 최적의 종목조합을 찾는데 활용할 종목조건 변수를 기존 6개의 종목조건 변수 중에서 선택합니다.
3. 변수별로 20% 단위로 가능한 종목 필터링을 모두 실시하여 최적의 종목조합을 그리드 서치합니다.
4. 그리드 서치 시 최적의 종목조합을 찾는 기준 지표는 탈출률, 수익률 두가지를 활용합니다.
5. 그리드 서치 시 원하는 변수를 추가할 수 있고 그리드서치 단위도 20%에서 더 크거나 작게 조절 할 수 있습니다.
5. 그리드 서치 결과 중 탈출률 혹은 수익률 상위 20개 종목조합을 표로 출력합니다.

**매수-매도 조건과 거시변수 조합 조건에 따른 최적의 종목조합을 찾는 그리드 서치 함수의 코드**

In [ ]:
# _1. Package load ------------------------------------------------------------
#library(plotly) 
library(tidyverse)
library(tidyquant)
library(magrittr)
library(inspectdf) 
library(tidymodels) 
library(cowplot) 
library(timetk) 
library(data.table)
library(tictoc) 
library(lubridate)
library(collapse)
library(ggTimeSeries)
#library(fmsb)
#devtools::install_github("ricardo-bion/ggradar",dependencies = TRUE)
#library(ggradar)
library(DBI)
library(RSQLite)
library(furrr)
library(zeallot)
library(gtsummary)
library(gt)


# _2. Data ,code load ---------------------------------------------------------------

future::plan(workers = parallel::detectCores()-1,strategy = "multisession")


n_day_data<- fread("E:\\trading_2019_to_2021.csv", encoding = 'UTF-8')
n_day_data %>% 
    fmutate(종목코드 = sprintf("%06d",종목코드) ) %>% 
    left_join(readRDS(file = "E:\\new_variable_daily_fluc.rds"),
              by = c("종목코드","open_date"="거래일자")) %>% 
    fmutate(open_date = ymd(open_date)) ->n_day_data
#source("C:/Users/shj/Documents/2022_summer_project/krx_repo/0366_seongho/03_scripts/back_test_function.R")



# 스팩주 두개가 이름에 스팩없는상태여서 필터링이 안됐음. 이거 삭제해야할듯.
# 추가적으로 투자자보호를 위해 관리종목, 투자주의 환기종목은 필터링에서 제외할까?
# n_day_data %>% 
#   filter(!(소속부 %in% c("관리종목(소속부없음)","SPAC(소속부없음)")))

macro_filter <- fread("E:\\macro_filter.csv", encoding = 'UTF-8')
esg.data <- read_csv("E:\\esg_table.csv")
esg.data %>% 
    select(2,4) %>%
    filter(nchar(ESG등급)>0)->esg.data

# ESG 합치기
n_day_data <- merge.data.table(n_day_data, esg.data, by.x="종목명", by.y="기업명", all.x=T)

n_day_data %<>% 
    fmutate(ESG등급 = ifelse(is.na(ESG등급),"미지정",ESG등급) %>%as_factor()) %>% 
    fmutate(ESG등급 = ESG등급 %>% fct_relevel("A+","A","B+","B","C","D","미지정") )

n_day_data$ESG등급 %>% levels()
# ECOS 데이터 합치기

n_day_data <-merge.data.table(n_day_data, macro_filter, by.x="open_date", by.y="time", all.x=T)




# _3. 필터링을 위한 변수 계산 ------------------------------------------------------------------
return_rank_percentile<- function(x,invesrse = FALSE){
    if(invesrse == TRUE){
        x <- -x
    }
    x <- (frank(x,ties.method ="min")/length(x))*100 # 작은값일수록 순위가 높음. (= 최소값이 1등)
    return(x)
}



n_day_data |>
    # 변수만드는데 필요한 계산 먼저 완료.
    fmutate(시가총액 = 시가총액 / 100000000)  |>  #overflow문제있는것같아서 억단위로 바꿈.영향 없음.
    fgroup_by(종목코드)  |> 
    fmutate(거래량_lag = flag(거래량,1))  |> 
    fmutate(고가_lag = flag(고가, 1))  |> 
    fmutate(roll_mean_30_거래= frollmean(x = 거래량_lag,n = 30,align = "right",fill = 1) )  |>  
    fmutate(변동성= 거래량/roll_mean_30_거래)  |>  
    fmutate(`최근20거래일고가` = frollapply(고가_lag, 20, max))  |> 
    fmutate(`20거래일고가대비등락률` = (종가-최근20거래일고가)/종가) |>  
    fungroup() |>
    fgroup_by(open_date) |>
    fmutate(new_PER = ifelse(is.na(PER),0,1/PER)) |>
    fmutate(유동성 = 거래량/상장주식수) |>
    
    # 계산된 변수들에 percentile 반환하는 함수 일괄 적용
    fmutate(across(.cols = c(시가총액,new_PER,유동성,변동성, `20거래일고가대비등락률`,일중수익률변동성),
                   .fns = function(x)return_rank_percentile(x,invesrse = TRUE),
                   .names = function(.cols,f) paste0(.cols,"_percentile") )) |>
    fungroup() %>%
    fselect(names(.) %>% grep(pattern ="open_date|종목|종가|거래량|소속부|시장구분|momentum|_percentile|ESG")) |>
    # 앞에 NA 자르고, 2022-06-30 까지 있으니까 현재 기준으로 필터링은 되지만 아직 탈출률 결정안된 날들이 있음
    # 일중데이터를 통한 보완은 1분기밖에 없어서 못했다고하면 될듯.
    # 깔끔하게 2019-07-01 ~ 2022-06-30 3년데이터
    filter(open_date >="2019-07-01") ->  processed_data

#processed_data %>% write_csv("processed_data.csv")

# chance=0.1,minus=-0.03,a=0,obs_period=30 인 결과 불러오기
result_back_test <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.1,minus=-0.05,a=0,obs_period=30.rds")


# 그리드서치 함수-------------------------------------------------------------------------------------------

# 필터링을 위해 최소한 거래량이 있는 종목을 선정한다. 또한, 관리종목대상으로는 투자하지 않기로한다.
processed_data |> 
    filter(거래량>0) |> 
    filter(!(소속부 %in% c("관리종목(소속부없음)","투자주의환기종목(소속부없음)","SPAC(소속부없음)"))) ->
    processed_data_modify


processed_data_modify<- left_join(processed_data_modify,result_back_test %>%
                                      select(filtered_date,종목코드,contains("first"),contains("실제"),final_price, 구매),
                                  by=c("open_date"="filtered_date","종목코드"))


# 그리드 함수 만들기
# 종목별 필터링에 대한 그리드(stock_grid_final)

# 0~100 사이를 20단위로 분할
stock_grid <- expand.grid(
    # PER.L      = seq(0,100, by= 10),
    # PER.U      = seq(0,100, by= 10),
    # 시가총액.L = seq(0,100, by= 10),
    # 시가총액.U = seq(0,100, by= 10))
     유동성.L   = seq(0,100, by= 20),
     유동성.U   = seq(0,100, by= 20),
    # 변동성.L   = seq(0,100, by= 25),
    # 변동성.U   = seq(0,100, by= 25),
    `20거래일고가대비등락률.L`   = seq(0,100, by= 20),
    `20거래일고가대비등락률.U`  = seq(0,100, by= 20),
     일중수익률변동성.L   = seq(0,100, by= 20),
     일중수익률변동성.U   = seq(0,100, by= 20))

gc()

stock_grid |> 
#     filter(PER.L < PER.U)  |> 
#     filter(시가총액.L < 시가총액.U)
     filter(유동성.L < 유동성.U) |> 
#     filter(변동성.L < 변동성.U) |> 
     filter(`20거래일고가대비등락률.L` < `20거래일고가대비등락률.U`) |>
     filter(일중수익률변동성.L < 일중수익률변동성.U) -> stock_grid_final



# 10~90 사이를 20단위로 분할
# stock_grid_10<- expand.grid(
#     PER.L      = seq(10,90, by= 20),
#     PER.U      = seq(10,90, by= 20),
#     시가총액.L = seq(10,90, by= 20),
#     시가총액.U = seq(10,90, by= 20),
#     유동성.L   = seq(10,90, by= 20),
#     유동성.U   = seq(10,90, by= 20),
#     변동성.L   = seq(10,90, by= 20),
#     변동성.U   = seq(10,90, by= 20),
#     20거래일고가대비등락률.L   = seq(10,90, by= 20),
#     20거래일고가대비등락률.U   = seq(10,90, by= 20),
#     일중수익률변동성.L   = seq(10,90, by= 20),
#     일정수익률변동성.U   = seq(10,90, by= 20))

gc()

# stock_grid_10 %>%  
#     filter(PER.L < PER.U)  |>
#     filter(시가총액.L < 시가총액.U) |>
#     filter(유동성.L < 유동성.U) |> 
#     filter(변동성.L < 변동성.U) |>
#     filter(20거래일고가대비등락률.L < 20거래일고가대비등락률.U) |>
#     filter(일중수익률변동성.L < 일중수익률변동성.U)-> stock_grid_modify_10

# 만들어둔 두 개의 그리드 합치기
# stock_grid_final <- rbind(stock_grid_modify_20, stock_grid_modify_10)


# 콜금리 +, 소비자물가지수 +, 국제유가 -, 뉴스심리지수 +, 금 - 일때를 가정하자
processed_data_modify |> 
        #____확인하고 싶은 날짜 필터링----------------------
filter(open_date %>% between(lower = "2019-07-01", upper = "2022-06-30")) %>%  
    {.->> selected_date} %>% 
    filter(`콜금리(익일물)(연%) _momentum` %in% c(-1,1)) %>%
    filter(`회사채수익률(3년,AA-)(연%) _momentum` %in% c(-1,1)) %>% 
    filter(`원/달러 환율(종가)(원) _momentum` %in% c(-1,1)) %>%  
    filter(`소비자물가지수(2020=100) _momentum` %in% c(-1)) %>%  
    filter(`국제유가(Dubai, 현물)(U$/bbl) _momentum` %in% c(-1,1)) %>%  
    filter(`뉴스심리지수() _momentum` %in% c(-1,1)) %>%
    filter(`원/위안 환율(종가)(원) _momentum` %in% c(-1,1)) %>%
    filter(`금($/OZ) _momentum` %in% c(-1)) %>%
    select(open_date, 종목코드, 시장구분, ESG등급,
        contains("percentile"), contains("first"),
        contains("실제"),final_price, 구매) -> ecos_filtered_data


ending_return_table <- data.frame(matrix(nrow = nrow(stock_grid_final), ncol = 4))
colnames(ending_return_table) <- c("유동성 상위(%)", "20거래일고가대비등락률 상위(%)", "일중수익률변동성 상위(%)", "평균수익률")

ending_escape_table <- data.frame(matrix(nrow = nrow(stock_grid_final), ncol = 4))
colnames(ending_escape_table) <- c("유동성 상위(%)", "20거래일고가대비등락률 상위(%)", "일중수익률변동성 상위(%)", "평균탈출률")


for(j in 1:nrow(stock_grid_final)){
    print(j)
    # PER.L      = stock_grid_final$PER.L[j]
    # PER.U      = stock_grid_final$PER.U[j]
    # 시가총액.L = stock_grid_final$시가총액.L[j]
    # 시가총액.U = stock_grid_final$시가총액.U[j]
     유동성.L   = stock_grid_final$유동성.L[j]
     유동성.U   = stock_grid_final$유동성.U[j]
    # 변동성.L   = stock_grid_final$변동성.L[j]
    # 변동성.U   = stock_grid_final$변동성.U[j]
     `20거래일고가대비등락률.L`  = stock_grid_final$`20거래일고가대비등락률.L`[j]
     `20거래일고가대비등락률.U`  = stock_grid_final$`20거래일고가대비등락률.U`[j]
     일중수익률변동성.L  = stock_grid_final$일중수익률변동성.L[j]
     일중수익률변동성.U  = stock_grid_final$일중수익률변동성.U[j]
    
    
    # 탈출률
    ecos_filtered_data |>
        # 거래 정보 필터링
        # filter(new_PER_percentile %>% between(lower = PER.L, upper = PER.U ))|>
        filter(시가총액_percentile %>% between(lower = 0, upper = 100))|>
        filter(유동성_percentile %>% between(lower = 유동성.L, upper = 유동성.U)) |>
        # filter(변동성_percentile %>% between(lower = 변동성.L, upper = 변동성.U)) |>
        filter(`20거래일고가대비등락률_percentile` %>% between(lower = `20거래일고가대비등락률.L`, upper = `20거래일고가대비등락률.U`)) |>
        filter(일중수익률변동성_percentile %>% between(lower = 일중수익률변동성.L, upper = 일중수익률변동성.U)) |>
        fmutate(탈출여부 = ifelse(탈출_first == 10000, 0, 1)) |>
        filter(구매==1) |>
        fgroup_by(open_date) |>
        fsummarise(mean_escape = mean(탈출여부,na.rm=TRUE)) -> result_stock_grid_escape
    
    
    ending_escape_table[j,1] <- str_glue("{유동성.L}~{유동성.U}%")
    ending_escape_table[j,2] <- str_glue("{`20거래일고가대비등락률.L`}~{`20거래일고가대비등락률.U`}%")
    ending_escape_table[j,3] <- str_glue("{일중수익률변동성.L}~{일중수익률변동성.U}%")
    ending_escape_table[j,4] <- str_glue("{round(mean(result_stock_grid_escape$mean_escape),3)*100}%")
    
    
    # 수익률
    ecos_filtered_data |>
        # 거래 정보 필터링
        # filter(new_PER_percentile %>% between(lower = PER.L, upper = PER.U ))|>
        filter(시가총액_percentile %>% between(lower = 0, upper = 90))|>
        filter(유동성_percentile %>% between(lower = 유동성.L, upper = 유동성.U)) |>
        # filter(변동성_percentile %>% between(lower = 변동성.L, upper = 변동성.U)) |>
        filter(`20거래일고가대비등락률_percentile` %>% between(lower = `20거래일고가대비등락률.L`, upper = `20거래일고가대비등락률.U`)) |>
        filter(일중수익률변동성_percentile %>% between(lower = 일중수익률변동성.L, upper = 일중수익률변동성.U)) |>
        fmutate(ending_position = fifelse(탈출_first==10000 &손절_first==10000, "최종가매도",
                                             if_else(탈출_first !=10000,"탈출","손절")) |>as_factor()) |>
        fmutate(ending_return = fifelse(ending_position =="최종가매도",
                                        final_price/실제구매가-1 ,
                                        if_else(ending_position=="탈출",
                                                실제익절가_first/실제구매가-1,
                                                실제손절가_first/실제구매가-1))) |>
        filter(구매==1) |>
        fgroup_by(open_date) |>
        fsummarise(mean_return = mean(ending_return)) -> result_stock_grid_return
    
    ending_return_table[j,1] <- str_glue("{유동성.L}~{유동성.U}%")
    ending_return_table[j,2] <- str_glue("{`20거래일고가대비등락률.L`}~{`20거래일고가대비등락률.U`}%")
    ending_return_table[j,3] <- str_glue("{일중수익률변동성.L}~{일중수익률변동성.U}%")
    ending_return_table[j,4] <- str_glue("{round(mean(result_stock_grid_return$mean_return),3)*100}%")
}                               




# 내림차순 정렬

ending_escape_table[ending_escape_table == 'NA%'] <- 0
ending_escape_table <- ending_escape_table[order(ending_escape_table[,4], decreasing = T),]

ending_escape_table_high_20 <- ending_escape_table[1:20, 1:4]


ending_return_table[ending_return_table == 'NA%'] <- 0
ending_return_table <- ending_return_table[order(ending_return_table[,4], decreasing = T),]

ending_return_table_high_20 <- ending_return_table[1:20, 1:4]


# gt package 활용해서 표로 출력
ending_escape_table_high_20 %>% 
    gt() %>%
    tab_header(title = "종목필터링 조합 별 평균 탈출률 상위 20개 종목")

ending_return_table_high_20 %>% 
    gt() %>%
    tab_header(title = "종목필터링 조합 별 평균 수익률 상위 20개 종목")




# 그리드 서치 함수 끝----------------------------------------------------------------------------------------

**매수-매도 조건과 거시변수 조합 조건에 따른 최적의 종목조합을 찾는 그리드 서치 함수 활용 예제**

매수-매도 조건 : 목표수익률 = 0.1, 최대손실률 = -0.05, 목표구매가 = 0,obs_period = 30

거시변수 조합 조건  : 소비자물가지수 모멘텀 = -, 금 모멘텀 = - 

위와 같은 매수-매도 조건, 거시변수 조합 조건에 대한 최적의 종목 조합을 탐색합니다

1.   탈출률 상위 20개 종목 조합 표
<p align="center"> <img src = https://drive.google.com/uc?id=1vSmk-bhUquUXpmCnBaMPEel6qmgJa01F>


2.   수익률 상위 20개 종목 조합 표
<p align="center"> <img src = https://drive.google.com/uc?id=1Rzf_g05wXMFhwYNIF23qPSY8lk604Mgw>


### 7-2. 자동매매

전략검증이 잘 되더라도 실제로 이걸 투자에 반영하기에는 사람이 일일이 예약 주문을 할 수가 없다. 전략을 찾으면 그대로 실행해 보고 싶은 사람이 있을 수 있기 때문에 꼭 필요한 서비스이지만 어떻게 구현이 가능한지까지만 조사해 봤습니다. 



[한국투자증권 API를 이용한 트레이딩 with R]
https://m.blog.naver.com/leebisu/222704181327

### 7-3. 설정해둔 전략에 대한 정기 보고서 발간


나만의 전략 저장 기능을 추가하여 직접 테스트해 본 여러 전략들에 대해 정기적으로 결과를 요약하여 보고서로 배포


<p align="center"> <img src = https://drive.google.com/uc?id=18nF4m03rs7HSwpggsr2mJJTAHLpoI1nz>

### 7.4 수정주가 적용

 수정 주가,수정 거래량,수정 상장주식수 계산

 - 수정주가를 계산할 때, 일별데이터를 하고나니, 일중데이터에도 해야한다는 사실을 인지했습니다. 또한 기준가가 변하는 상황에 따라서 수정거래량, 수정 상장주식수를 계산하는 방식에 어려움이 있어 일별 데이터로 수정주가 계산하는 코드만 부록에 남기고 실제로 활용하지는 못했습니다. 



### 7.5 매도조건 다각화

현재는 수익률 기준으로 목표수익률, 최대손실폭 도달시 매도하고 있지만,
거시변수 상황이나 종목조건에 따라 매도전략을 설정할 수 있습니다.


# 부록

### 데이터 크롤링 및 전처리

**trading_2019_2022.csv 크롤링 코드**

코드전체를 실행하면 결과창이 너무 길어지기 때문에 예시로 1주일치 거래데이터를 가져오는 코드를 실행하였습니다.

두물머리의 이현열님께서 집필하신 'R을 이용한 퀀트 투자 포트폴리오 만들기(개정판)'의 도움을 받았습니다. 감사합니다.


R을 이용한 퀀트 투자 포트폴리오 만들기(개정판)[https://hyunyulhenry.github.io/quant_cookbook/%EA%B8%88%EC%9C%B5-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%88%98%EC%A7%91%ED%95%98%EA%B8%B0-%EA%B8%B0%EB%B3%B8.html]

In [ ]:
library(httr)
library(rvest)
library(tidyverse)
library(lubridate)
library(tictoc)

otp_url = "http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd"
csv_url = "http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd"



start_date<-'20190101' 
end_date<-  '20190108'  #출력이 너무 길어지므로 예시로 1주일치 거래데이터만 합치기.     #'20220630'
binded_data <- NULL

# 개별종목 크롤링을 통한 개장날짜 긁어오기 --------------------------------------------------
otp_form_data_date = list(
  tboxisuCd_finder_stkisu0_7= '005930/삼성전자',
  isuCd= 'KR7005930003',
  isuCd2= 'KR7005930003',
  codeNmisuCd_finder_stkisu0_7= '삼성전자',
  param1isuCd_finder_stkisu0_7= 'ALL',
  strtDd= start_date,
  endDd= end_date,
  share= '1',
  money= '1',
  csvxls_isNo= 'false',
  name= 'fileDown',
  url= 'dbms/MDC/STAT/standard/MDCSTAT01701'
)

otp_all_things_date = POST(otp_url, query = otp_form_data_date) %>% read_html() %>% html_text()
all_date <-  POST(csv_url, query = list(code = otp_all_things_date)) %>% 
  read_html(encoding = 'EUC-KR') %>% html_text() %>% read_csv() 
duration<-gsub('\\D','', all_date$일자)
duration <- rev(duration)

# 1.전종목기본정보 -----------------------------------------------------------------

all_things_kibon_form = list(
  mktId = "ALL",
  share = "1",
  csvxls_isNo = "false",
  name = "fileDown",
  url = "dbms/MDC/STAT/standard/MDCSTAT01901"
)

all_things_kibon_download = POST(otp_url, query = all_things_kibon_form) %>% read_html() %>% html_text()
all_things_kibon = POST(csv_url, query = list(code =all_things_kibon_download )) %>% 
  read_html(encoding = 'EUC-KR') %>% html_text() %>% read_csv() 


# open_date <- duration[1]
tic()

for(open_date in duration){
  # 2.전종목 시세 ------------------------------------------------------------------
  
  all_things_sise_form = list(
    mktId= 'ALL',
    trdDd= open_date,
    #adjStkPrc_check="Y", 혹시나해서 넣어봤는데 수정주가 적용안됨
    share= '1',
    money= '1',
    csvxls_isNo= 'false',
    name= 'fileDown',
    url= 'dbms/MDC/STAT/standard/MDCSTAT01501'
  )
  
  all_things_sise_download = POST(otp_url, query = all_things_sise_form) %>% read_html() %>% html_text()
  all_things_sise = POST(csv_url, query = list(code = all_things_sise_download)) %>% 
    read_html(encoding = 'EUC-KR') %>% html_text() %>% read_csv() #%>% mutate(time=Sys.time())
  #data_sise2 = POST(csv_url, query = list(code = otp1)) %>% 
  #read_html(encoding = 'EUC-KR') %>% html_text() %>% read_csv() %>% mutate(time=Sys.time())
  
  # 5.투자지표들 -------------------------------------------------------------------
  investment_index_form = list(
    searchType='1',
    mktId='ALL',
    trdDd=open_date,
    tboxisuCd_finder_stkisu0_0='',
    isuCd='',
    isuCd2='', 
    codeNmisuCd_finder_stkisu0_0=' ',
    param1isuCd_finder_stkisu0_0='ALL',
    strtDd='',
    endDd='', 
    csvxls_isNo='false',
    name='fileDown',
    url='dbms/MDC/STAT/standard/MDCSTAT03501'
  )
  
  investment_index_download = POST(otp_url, query = investment_index_form) %>% read_html() %>% html_text()
  investment_index = POST(csv_url, query = list(code = investment_index_download)) %>% 
    read_html(encoding = 'EUC-KR') %>% html_text() %>% read_csv() # 날짜 추가 
  
  # 7.외국인보유비중 -----------------------------------------------------------------
  foreinger_proportion_form = list(
    searchType='1',
    mktId='ALL',
    trdDd=open_date,
    tboxisuCd_finder_stkisu0_2='',
    isuCd='',
    isuCd2='', 
    codeNmisuCd_finder_stkisu0_2=' ',
    param1isuCd_finder_stkisu0_2='ALL',
    strtDd='',
    endDd='', 
    csvxls_isNo='false',
    name='fileDown',
    url='dbms/MDC/STAT/standard/MDCSTAT03701'
  )
  foreinger_proportion_download = POST(otp_url, query = foreinger_proportion_form) %>% read_html() %>% html_text()
  foreinger_proportion = POST(csv_url, query = list(code = foreinger_proportion_download)) %>% 
    read_html(encoding = 'EUC-KR') %>% html_text() %>% read_csv() # 날짜 추가
  # 9.업종분류현황 ------------------------------------------------------------------
  if(as.numeric(open_date)>=20020102){
    listed_industry_distribution_kospi_form = list(
      mktId= 'STK', 
      trdDd= open_date,
      money= '1',
      csvxls_isNo= 'false',
      name= 'fileDown',
      url= 'dbms/MDC/STAT/standard/MDCSTAT03901'
    )
    
    listed_industry_distribution_kospi_download = POST(otp_url, query = listed_industry_distribution_kospi_form) %>% read_html() %>% html_text()
    listed_industry_distribution_kospi = POST(csv_url, query = list(code = listed_industry_distribution_kospi_download)) %>% 
      read_html(encoding = 'EUC-KR') %>% html_text() %>% read_csv() 
    
    listed_industry_distribution_kosdaq_form = list(
      mktId= 'KSQ', #코스닥만 불러오게됨, 코스피=STK
      trdDd= open_date,
      money= '1',
      csvxls_isNo= 'false',
      name= 'fileDown',
      url= 'dbms/MDC/STAT/standard/MDCSTAT03901'
    )
    
    listed_industry_distribution_kosdaq_download = POST(otp_url, query = listed_industry_distribution_kosdaq_form) %>% read_html() %>% html_text()
    listed_industry_distribution_kosdaq = POST(csv_url, query = list(code = listed_industry_distribution_kosdaq_download)) %>% 
      read_html(encoding = 'EUC-KR') %>% html_text() %>% read_csv() 
    
    
    all_things_industry<- bind_rows(listed_industry_distribution_kosdaq,listed_industry_distribution_kospi)
    wrangled_all_things_industry<- all_things_industry %>% select(1,2,4) 
  }
  
  
  
  # total_data 로 통합 ---------------------------------------------------------
  
  wrangled_all_things_sise<-all_things_sise[!grepl('스팩', all_things_sise$종목명), ]  
  wrangled_all_things_sise<-wrangled_all_things_sise %>%
    filter(as.numeric(종목코드)%%10==0, 시장구분!="KONEX"
           #,거래량>0
    ) 
  
  wrangled_all_things_kibon <- all_things_kibon %>% 
    select(2,4,6) %>% 
    mutate(상장일 = as.character(상장일)) %>% 
    purrr::set_names(all_things_sise %>% select(1:2) %>% colnames(),"상장일")
  
  
  wrangled_investment_index <- investment_index %>% select(1,2,6,7,10:13)
  wrangled_foreinger_proportion <- foreinger_proportion %>% select(1,2,7:10) %>% 
    purrr::set_names(c("종목코드","종목명","외국인보유수량","외국인지분율","외국인한도수량","외국인한도소진율"))
  
  total_data <- left_join(wrangled_all_things_sise,wrangled_all_things_kibon ,c("종목코드","종목명"))
  total_data <- left_join(total_data,wrangled_investment_index,c("종목코드","종목명"))
  total_data <- left_join(total_data,wrangled_foreinger_proportion,c("종목코드","종목명"))  
  ifelse(as.numeric(open_date)>=20020102,
         total_data <- left_join(total_data,wrangled_all_things_industry,c("종목코드","종목명")),
         total_data <- total_data %>% mutate(업종명= NA)
  )
  
  total_data <- total_data %>% mutate(open_date= open_date)
  binded_data <- bind_rows(binded_data,total_data) 
  print(open_date)
  
}
toc()

#write_csv(binded_data,"trading_2019_to_2022.csv")
binded_data %>% head()

# 스팩주, 우선주 , KONEX시장 제외함.


Rows: 5 Columns: 11
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
dbl  (10): 종가, 대비, 등락률, 시가, 고가, 저가, 거래량, 거래대금, 시가총액, 상장주식수...
date  (1): 일자

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2631 Columns: 12
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr  (10): 표준코드, 단축코드, 한글 종목명, 한글 종목약명, 영문 종목명, 시장구분, 증권구분, 소속부, 주식종류, 액면가...
dbl   (1): 상장주식수
date  (1): 상장일

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2379 Columns: 14
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr  (4): 종목코드, 종목명, 시장구분, 소속부

[1] "20190102"


Rows: 2379 Columns: 14
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr  (4): 종목코드, 종목명, 시장구분, 소속부
dbl (10): 종가, 대비, 등락률, 시가, 고가, 저가, 거래량, 거래대금, 시가총액, 상장주식수...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2187 Columns: 13
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr (2): 종목코드, 종목명
dbl (9): 종가, 대비, 등락률, EPS, PER, BPS, PBR, 주당배당금, 배당수익률
lgl (2): 선행 EPS, 선행 PER

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2379 Columns: 10
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr (2): 종목코드, 종목명
dbl 

[1] "20190103"


Rows: 2379 Columns: 14
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr  (4): 종목코드, 종목명, 시장구분, 소속부
dbl (10): 종가, 대비, 등락률, 시가, 고가, 저가, 거래량, 거래대금, 시가총액, 상장주식수...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2187 Columns: 13
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr (2): 종목코드, 종목명
dbl (9): 종가, 대비, 등락률, EPS, PER, BPS, PBR, 주당배당금, 배당수익률
lgl (2): 선행 EPS, 선행 PER

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2379 Columns: 10
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr (2): 종목코드, 종목명
dbl 

...

Rows: 2379 Columns: 14
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr  (4): 종목코드, 종목명, 시장구분, 소속부
dbl (10): 종가, 대비, 등락률, 시가, 고가, 저가, 거래량, 거래대금, 시가총액, 상장주식수...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2187 Columns: 13
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr (2): 종목코드, 종목명
dbl (9): 종가, 대비, 등락률, EPS, PER, BPS, PBR, 주당배당금, 배당수익률
lgl (2): 선행 EPS, 선행 PER

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2379 Columns: 10
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr (2): 종목코드, 종목명
dbl 

[1] "20190107"


Rows: 2379 Columns: 14
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr  (4): 종목코드, 종목명, 시장구분, 소속부
dbl (10): 종가, 대비, 등락률, 시가, 고가, 저가, 거래량, 거래대금, 시가총액, 상장주식수...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2187 Columns: 13
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr (2): 종목코드, 종목명
dbl (9): 종가, 대비, 등락률, EPS, PER, BPS, PBR, 주당배당금, 배당수익률
lgl (2): 선행 EPS, 선행 PER

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2379 Columns: 10
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr (2): 종목코드, 종목명
dbl 

[1] "20190108"
16.23 sec elapsed


종목코드,종목명,시장구분,소속부,종가,대비,등락률,시가,고가,저가,⋯,BPS,PBR,주당배당금,배당수익률,외국인보유수량,외국인지분율,외국인한도수량,외국인한도소진율,업종명,open_date
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
060310,3S,KOSDAQ,중견기업부,2190,-25,-1.13,2280,2280,2160,⋯,704,3.11,0,0.00,925605,2.08,44395878,2.08,기계·장비,20190102
095570,AJ네트웍스,KOSPI,NA,4530,25,0.55,4530,4605,4445,⋯,6168,0.73,86,1.90,3743075,7.99,46822295,7.99,서비스업,20190102
068400,AJ렌터카,KOSPI,NA,12350,50,0.41,12750,12950,12250,⋯,11091,1.11,0,0.00,1202974,5.43,22146300,5.43,서비스업,20190102
006840,AK홀딩스,KOSPI,NA,51900,-2100,-3.89,53900,54000,51300,⋯,52954,0.98,650,1.25,2009793,15.17,13247561,15.17,기타금융,20190102
054620,APS홀딩스,KOSDAQ,우량기업부,4205,-15,-0.36,4225,4290,4155,⋯,13639,0.31,0,0.00,2030553,9.96,20394221,9.96,금융,20190102
265520,AP시스템,KOSDAQ,우량기업부,21500,-200,-0.92,21850,22150,21050,⋯,5856,3.67,0,0.00,1057649,7.50,14099511,7.50,반도체,20190102


### 한국은행 ECOS API를 이용한 100대 통계지표, 뉴스심리 지수 가져오기(원/위안 ,금 변수 추가)

In [ ]:
#if (!require(devtools)) install.packages("devtools"); require(devtools)  
#devtools::install_github("seokhoonj/ecos", force = TRUE)  패키지 설치해야됨. 주석제거후 실행.

library(tidyverse)
library(ecos)
library(lubridate)
library(zoo)
library(timetk)

# 인증키는 ECOS 사이트 https://ecos.bok.or.kr/api/  가서 회원가입 후 발금.
#usethis::edit_r_environ() #API발급된 key 저장(보안을 위해 따로 저장후 로드)
my_key <- Sys.getenv("ECOS_API_KEY") 
keyStatList<- ecos::keyStatList(my_key)



Please get your api key from website 'https://ecos.bok.or.kr/api/#/AuthKeyApply'



In [ ]:
# 통계코드,아이템 코드 찾아서 입력 ------------------------------------------------------

# 100대지표에 대해서 전부 stat_code, item_code를 찾고나서 다른사람들이 사용하기 편하게 
# 공개하고자 하였으나 잘 찾을 수 없는 데이터가 있어서 보류.

stat_code <-  c("722Y001",
                rep("817Y002",7),
                "121Y002"       , 
                "121Y006"       ,
                "104Y015"       ,
                "104Y016"       ,
                "151Y001"       ,
                "901Y054"       ,
                "101Y019"       , #14
                "101Y004"       ,
                "111Y004"       ,
                "112Y002"       ,
                "731Y003"       ,
                "731Y001"       ,
                "731Y001"       ,
                "731Y003"       ,
                "802Y001"       ,
                "802Y001"       ,
                "901Y014"       , 
                "901Y056"       , #26 고객예탁금.
                "901Y015"       ,
                "191Y001"       ,
                "902Y015"       ,
                "200Y002"       ,
                "200Y002"       ,
                "200Y002"       ,
                "200Y002"       ,
                "200Y003"       ,
                NA              , #1인당GNI
                "200Y002"       ,
                "200Y002"       ,
                "200Y002"       ,
                "901Y033"       ,
                "901Y032"       ,
                "901Y032"       ,
                "901Y032"       ,
                "901Y035"       ,
                "901Y038"       ,
                NA              , #도소매업지수
                NA              , #소매판매액지수
                "601Y003"       ,
                "901Y100"       ,
                "901Y066"       ,
                "901Y036"       ,
                "901Y018"       ,
                "901Y104"       ,
                "901Y037"       ,
                "901Y020"       ,
                NA              , #901Y103인데 구체적으로 어떤값인지 확실치 않음.
                "901Y067"       ,
                "901Y067"       ,
                "511Y002"       ,
                "512Y007"       ,
                "513Y001"       ,
                "501Y005"       ,
                NA              , #제조업매출액세전순이익률
                NA              , #제조업부채비율
                NA              , #가구당월평균소득
                NA              , #평균소비성향
                NA              , #지니계수
                NA              , #5분위배율
                NA              , #실업률
                NA              , #고용률
                NA              , #경제활동인구
                NA              , #취업자수
                NA              , #시간당명목임금지수
                NA              , #노동생산성지수
                NA              , #단위노동비용지수
                NA              , #추계인구
                NA              , #고령인구비율(65세이상)
                "901Y028"       , #합계출산율
                NA              , #경상수지
                NA              , #직접투자(자산)
                NA              , #직접투자(부채)
                NA              , #증권투자(자산)
                NA              , #증권투자(부채)
                NA              , #수출금액지수
                NA              , #수입금액지수
                NA              , #순상품교역조건지수
                NA              , #소득교역조건지수
                NA              , #외환보유액
                NA              , #대외채무
                NA              , #대외채권
                "901Y009"       , #소비자물가지수
                NA              , #농산물 및 석유류제외 소비자물가지수
                NA              , #생활물가지수
                NA              , #생산자물가지수
                NA              , #수출물가지수
                NA              , #수입물가지수
                NA              , #주택매매가격지수
                NA              , #주택전세가격지수
                NA              , #지가변동률(전기대비)
                "902Y003"       , #국제유가(Dubai,현물)
                "902Y003"         #금
                
)


item_code1 <- c("0101000"  , #한국은행 기준금리
                "010101000", #콜금리
                "010150000", #KORIBOR(3개월)
                "010502000", #CD수익률(91일),
                "010400001", #통안증권(1년),
                "010200000", #국고채수익률(3년)
                "010200001", #국고채수익률(5년)
                "010300000", #회사채수익률(3년,AA-)
                "BEABAA2"  , #예금은행 수신금리 -저축성수신
                "BECBLA01" , #10 #예금은행 대출금리 -대출평균
                "BDAA1"    , #예금은행총예금(말잔)
                "BDCA1"    , #예금은행대출금(말잔)
                "1000000"  , #가계신용
                "MO3AB"    , #가계대출연체율 ,주기가 M인데 3개 값이 있음.group_by time했을때 1번째 index
                "BBLA00"   , #M1(협의통화, 평잔)
                "BBHA00"   , #M2(광의통화, 평잔)
                "LA00000"  , #Lf(평잔)
                "X000000"  , #L(말잔)
                "0000003"  , #원/달러 환율(종가)
                "0000002"  , #원/엔(100엔) 환율(매매기준율)
                "0000003"  , #원/유로 환율(매매기준율)
                "0000010"  , #원/위안 환율(종가),
                "0001000"  , #코스피지수
                "0089000"  , #코스닥지수
                "1060000"  , #주식거래대금 (월, 코스피만) *일별 거래대금도 코스피,코스닥 각각 얻을 수  있음.
                "S23A"     , #고객예탁금 
                "1"        , #채권거래대금 2040000이아님..
                "0200000"  , #국고채발행액 group_by time했을때 1번째 index
                "KOR"      , #경제성장률 
                "10122"    , #민간소비증감률(실질, 계절조정 전기대비)
                "10123"    , #설비투자증감률(실질, 계절조정 전기대비)
                "10124"    , #건설투자증감률(실질, 계절조정 전기대비)
                "10125"    , #재화의 수출 증감률(실질, 계절조정 전기대비)
                "1400"     , #GDP(명목, 계절조정),
                NA         , #1인당GNI
                "40101"    , #총저축률
                "40102"    , #국내총투자율
                "501"      , #수출입의 대 GNI 비율 (분기) #100대지표는 단위가 연인데, groupby 연도해서 평균하면 될듯.
                "A00"      , #전산업생산지수(농림어업제외) group_by time했을때 2번째 index
                "I11AC"    , #제조업생산지수 group_by time했을때 2번째 index. or item code2로접근해도됨.
                "I11AC"    , #제조업출하지수 group_by time했을때 4번째 index.
                "I11AC"    , #제조업재고지수 group_by time했을때 6번째 index.
                "I32A"     , #제조업가동률지수 group_by time했을때 3번째 index.
                "I51A"     , #서비스업생산지수 group_by time했을때 3번째 index
                NA         , #도소매업지수
                NA         , #소매판매액지수
                "201000"   , #개인신용카드사용액 group_by time했을때 1번째 index
                "G11"      , #자동차판매액지수 group_by time했을때 3번째 index
                "I15B"     , #설비투자지수
                "20"       , #기계류내수출하지수 , group_by time했을때 6번째 index=마지막
                "I35AAA"   , #국내수요기계수주액
                "I48A"     , #건설기성액 group_by time했을때 4번째 index=마지막
                "1"        , #건축허가면적 group_by time했을때 1번째 index
                "I42A"     , #건설수주액
                NA         , #건축물착공면적
                "I16D"     , #경기동행지수순환변동치
                "I16E"     , #경기선행지수순환변동치
                "FME"      , #소비자심리지수
                "C0000"    , #제조업업황실적BSI group_by time했을때 1번째 index
                "E1000"    , #경제심리지수
                "C"        , #제조업매출액증감률 group_by time했을때 18번째 index
                NA         , #제조업매출액세전순이익률
                NA         , #제조업부채비율
                NA         , #가구당월평균소득
                NA         , #평균소비성향
                NA         , #지니계수
                NA         , #5분위배율
                NA         , #실업률
                NA         , #고용률
                NA         , #경제활동인구
                NA         , #취업자수
                NA         , #시간당명목임금지수
                NA         , #노동생산성지수
                NA         , #단위노동비용지수
                NA         , #추계인구
                NA         , #고령인구비율(65세이상)
                "I35E"     , #합계출산율
                NA         , #경상수지
                NA         , #직접투자(자산)
                NA         , #직접투자(부채)
                NA         , #증권투자(자산)
                NA         , #증권투자(부채)
                NA         , #수출금액지수
                NA         , #수입금액지수
                NA         , #순상품교역조건지수
                NA         , #소득교역조건지수
                NA         , #외환보유액
                NA         , #대외채무
                NA         , #대외채권
                "0"        , #소비자물가지수
                NA         , #농산물 및 석유류제외 소비자물가지수
                NA         , #생활물가지수
                NA         , #생산자물가지수
                NA         , #수출물가지수
                NA         , #수입물가지수
                NA         , #주택매매가격지수
                NA         , #주택전세가격지수
                NA         , #지가변동률(전기대비)
                "4010102"  , #국제유가(Dubai,현물)
                "4020401"    #금
                
)


# 함수 1. cycle to code -----------------------------------------------------
# cycle 입력받으면 , D, M , A ,Q, S, SM 으로 반환하기.
# D	YYYYMMDD	20210917
# M	YYYYMM	202109
# Q	YYYYQq (q : 1～4)	2021Q3
# S	YYYYSs(s : 1～2)	2021S2
# A	YYYY	2021
# SM	YYYYMMSs(s : 1～2)	202101S1
cycle_to_code<- function(x) {
  case_when(
    (nchar(x) == 4)                      == TRUE ~ "A",
    (nchar(x) == 6) & str_detect(x, "S") == TRUE ~ "S",
    (nchar(x) == 6) & str_detect(x, "Q") == TRUE ~ "Q",
    (nchar(x) == 6)                      == TRUE ~ "M",
    (nchar(x) == 8) & str_detect(x, "S") == TRUE ~ "SM",
    TRUE ~ "D"
  )
}


# form --------------------------------------------------------------------

# 찾을 데이터의 시기는 2019년~2022년이므로 이거에 맞게 
cycle_time_form<- tribble(~cycle ,~start_time ,~end_time  ,
                          "D"    , "20190101" , "20220731", 
                          "M"    , "201901"   , "202207"  , 
                          "Q"    , "2019Q1"   , "2022Q2"  , 
                          "A"    , "2019"     , "2022"    ,
                          "S"    , "2019S1"   , "2022S1"  ,
                          "SM"   , "201901S1" , "202207S2"
)
# 이것도 추후에는 start_time, end_time 입력 하면 알아서 저렇게 변환되게 짜보기.

# 함수 2 .패딩하고 모멘텀 반환--------------------------------------------------------
padding_and_return_momentum <- function(data){
  data %>%
    mutate(across(.cols = everything(), .fns = ~na.locf(.x,na.rm = FALSE) )) %>% 
    mutate(across(.cols = last_col(), .fns = ~lag(.x) ,.names = "{.col}_lag1")) %>%  
    {.->>temp} %>% 
    mutate(momentum = .[[3]]-.[[4]]) %>% 
    mutate(momentum = ifelse(momentum ==0, NA,momentum)) %>% 
    mutate(momentum = na.locf(momentum,na.rm = FALSE)) %>% 
    mutate(momentum = ifelse(momentum <0, -1, 1)) ->data
  
  
}# lag된 변수는 제거하고 반환할까?

# 22, 100번이 각각 원/위안, 금 시세

purpose<- bind_cols(keyStatList, stat_code= stat_code,item_code1=item_code1) %>% slice(c(2,8,19,22,90,99,100))

# 화요일 오후 4시에 지수를 업로드하며, 국가통계가 아닌 점과 이용자
# 유의사항을 주석과 메타데이터에 수록
news<- tibble(class_name   = "뉴스심리지수",
              keystat_name = "뉴스심리지수",
              data_value   = "100"           ,
              cycle        = "20220101"      ,
              unit_name    = ""            ,
              stat_code    = "521Y001"     ,
              item_code1   = "A001"  
) 

purpose<- bind_rows(purpose,news)

purpose %>% 
  mutate(cycle_code = cycle_to_code(cycle)) -> purpose
purpose<- left_join(purpose,cycle_time_form ,by = c("cycle_code"="cycle"))
purpose %>% head(10) 

,class_name,keystat_name,data_value,cycle,unit_name,stat_code,item_code1,cycle_code,start_time,end_time
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,시장금리,콜금리(익일물),2.175,20220817,%,817Y002,010101000,D,20190101,20220731
2,시장금리,"회사채수익률(3년,AA-)",4.114,20220818,%,817Y002,010300000,D,20190101,20220731
3,환율,원/달러 환율(종가),1320.7,20220818,원,731Y003,0000003,D,20190101,20220731
4,환율,원/위안 환율(종가),193.87,20220818,원,731Y003,0000010,D,20190101,20220731
5,소비자/생산자 물가,소비자물가지수,108.74,202207,2020=100,901Y009,0,M,201901,202207
6,원자재,"국제유가(Dubai, 현물)",105.8,202207,달러,902Y003,4010102,M,201901,202207
7,원자재,금,1765.94,202207,달러,902Y003,4020401,M,201901,202207
8,뉴스심리지수,뉴스심리지수,100,20220101,,521Y001,A001,D,20190101,20220731


In [ ]:


all_day<- tibble(time = tk_make_timeseries(start_date = "2019-01-01",end_date = "2022-07-31"))
macro_momentum <- all_day # 최종 저장할 tibble(=데이터프레임)

for( i in 1:nrow(purpose)){
  parse_code<- paste0("ecos::statSearch(my_key,"           ,
                      "stat_code ='" ,  purpose$stat_code[i],
                      "',item_code1 ='", purpose$item_code1[i],
                      "',cycle ='",      purpose$cycle_code[i],
                      "',start_time = '",purpose$start_time[i],
                      "',end_time ='", purpose$end_time[i], "')")
  
  # 데이터 읽기.
  data <- eval(parse(text = parse_code)) %>%  select(tail(names(.), 3))  
  # 변수명 우리가 취한 데이터이름으로 변환 및 단위 붙여서 하나의 변수로 join예정
  # ( )는 nchar(data$unit_name[1])가 1이상인것에만 적용하게 수정할까.
  colnames(data)<- c("time",paste0(purpose$keystat_name[i],"(",data$unit_name[1],")"),"unit_name")
  data %>% 
    select(head(names(.),2)) %>%
    mutate(across(.cols = last_col(), .fns = ~lag(.x) ,.names = "{.col}_lag1")) ->data
  
  
  if(purpose$cycle_code[i] == "D"){
    
    data <- left_join(all_day,data %>% mutate(time = ymd(time)), by = "time")%>% 
      padding_and_return_momentum()
    
  }else if(purpose$cycle_code[i] == "M"){
    
    data <- left_join(all_day,data %>% mutate(time = ym(time)), by = "time")%>% 
      padding_and_return_momentum()
    
  }else if(purpose$cycle_code[i] == "Q"){
    
    data <- left_join(all_day,data %>% mutate(time = as.yearqtr(time) %>% as.Date ), by = "time")%>% 
      padding_and_return_momentum()
    
  }else if(purpose$cycle_code[i] == "A"){
    
    data <- left_join(all_day,data %>% mutate(time = make_date(year = time, month = 1, day = 1)) , by = "time") %>% 
      padding_and_return_momentum()
  }
  colnames(data) <-c(colnames(temp),paste(colnames(temp)[2],"_momentum"))
  macro_momentum <- left_join(macro_momentum,data,by = "time")
  
}


macro_momentum %>%
  select(time,contains("뉴스")) %>%
  select(1:2) %>% 
  mutate(day_of_week = lubridate::wday(time,label = T),.after=time) %>% 
  filter(day_of_week =="월") %>%
  tk_augment_lags(.value = contains("뉴스"),.lags = 1) %>% 
  {.->>temp} %>% 
  mutate(momentum = .[[3]]-.[[4]]) %>% 
  mutate(momentum = ifelse(momentum ==0, NA,momentum)) %>% 
  mutate(momentum = na.locf(momentum,na.rm = FALSE)) %>% 
  mutate(momentum = ifelse(momentum <0, -1, 1)) ->news_momentum

colnames(news_momentum) <-c(colnames(temp),paste(colnames(temp)[3],"_momentum"))

news_momentum<- left_join(all_day,news_momentum , by = "time") %>% 
  mutate(across(.cols = everything(), .fns = ~na.locf(.x,na.rm = FALSE) )) %>% 
  select(time,last_col()) %>% 
  mutate(`뉴스심리지수() _momentum` = lag(`뉴스심리지수() _momentum`,2))

macro_filter<- left_join(
  macro_momentum %>%
    select(-contains("뉴스")) %>%
    select(time,contains("momentum")) %>% 
    #콜금리는 익일물 정보를 알수 있으므로 lag한번 더 .
    mutate(`콜금리(익일물)(연%) _momentum` = lag(`콜금리(익일물)(연%) _momentum`)),
  news_momentum,
  by = "time"
)


macro_filter
#write_csv(macro_filter,"macro_filter.csv")




Using by: day



time,콜금리(익일물)(연%) _momentum,"회사채수익률(3년,AA-)(연%) _momentum",원/달러 환율(종가)(원) _momentum,원/위안 환율(종가)(원) _momentum,소비자물가지수(2020=100) _momentum,"국제유가(Dubai, 현물)(U$/bbl) _momentum",금($/OZ) _momentum,뉴스심리지수() _momentum
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2019-01-01,NA,NA,NA,NA,NA,NA,NA,NA
2019-01-02,NA,NA,NA,NA,NA,NA,NA,NA
2019-01-03,NA,NA,NA,NA,NA,NA,NA,NA
2019-01-04,NA,-1,1,1,NA,NA,NA,NA
2019-01-05,-1,-1,1,1,NA,NA,NA,NA
2019-01-06,-1,-1,1,1,NA,NA,NA,NA
2019-01-07,-1,-1,-1,-1,NA,NA,NA,NA
2019-01-08,-1,1,-1,-1,NA,NA,NA,NA
2019-01-09,-1,1,1,1,NA,NA,NA,NA


### 일중데이터를 통해 일중 수익률 변동성 변수 계산

In [ ]:


#현재의 작업공간에 있는 csv 파일의 목록 가져오기(recursive = T 로 하위폴더 포함)
files <-  dir(path = "C:/Users/shj/Documents/2022_summer_project/krx_repo",pattern="csv",recursive = T)
encoding<- "EUC-KR"
locale <- readr::locale(encoding = encoding)
files %>% tail(20)


[1] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201202.csv"
 [2] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201203.csv"
 [3] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201204.csv"
 [4] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201207.csv"
 [5] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201208.csv"
 [6] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201209.csv"
 [7] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201210.csv"
 [8] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201211.csv"
 [9] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201214.csv"
[10] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201215.csv"
[11] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201216.csv"
[12] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201217.csv"
[13] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201218.csv"
[14] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201221.csv"
[15] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201222.csv"
[16] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201223.csv"
[17] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201224.csv"
[18] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201228.csv"
[19] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201229.csv"
[20] "주식/[코스닥]일중 시세정보(1분)(주문번호-1418-6)/[코스닥]일중 시세정보(1분)(주문번호-1418-6)_20201230.csv"

In [ ]:

#연월 별로 파일을 불러와서 기존 trading_data와 결합
tk_make_timeseries(start_date = "2019-07-01",end_date = "2022-06-01") %>% enframe %>% 
  mutate(yearmonth = paste0(substr(value,1,4),substr(value,6,7))) %>% 
  group_by(yearmonth) %>% 
  group_keys()->date_key


n_day_data<- fread("C:/Users/shj/Documents/2022_summer_project/krx_repo/trading_2019_to_2022.csv",encoding="UTF-8")


n_day_data %>% 
  mutate(종목코드 = sprintf("%06d",종목코드)) %>% 
  group_by(종목코드) %>% 
  mutate(종가_lag = flag(종가,n = 1)) %>% ungroup() %>% 
  filter(open_date>=20190701) %>% 
  select(open_date, 종목명,시장구분,종목코드,종가_lag,거래량) ->n_day_data


n_day_data %>% 
  filter(거래량>0)

# 연월, 시장별로 따로 불러온뒤 결합하고 db저장, 다음 조합에대해 불러온뒤 다시결합하고 저장 반복.
db_case<- expand_grid(market_key = c("유가증권","코스닥"),
                      date_key)

db_case %>% head()
db_case %>% tail()

Using by: day



open_date,종목명,시장구분,종목코드,종가_lag,거래량
<int>,<chr>,<chr>,<chr>,<int>,<int>
20190701,3S,KOSDAQ,060310,2645,138345
20190701,AJ네트웍스,KOSPI,095570,5390,51445
20190701,AJ렌터카,KOSPI,068400,10450,238185
20190701,AK홀딩스,KOSPI,006840,44450,20570
20190701,APS홀딩스,KOSDAQ,054620,5840,370566
20190701,AP시스템,KOSDAQ,265520,26150,114800
20190701,AP위성,KOSDAQ,211270,6930,1908
20190701,BGF,KOSPI,027410,7400,84928
20190701,BGF리테일,KOSPI,282330,211000,13242


market_key,yearmonth
<chr>,<chr>
유가증권,201907
유가증권,201908
유가증권,201909
유가증권,201910
유가증권,201911
유가증권,201912


market_key,yearmonth
<chr>,<chr>
코스닥,202201
코스닥,202202
코스닥,202203
코스닥,202204
코스닥,202205
코스닥,202206


In [ ]:

#tictoc::tic()
##new_variable <- NULL
#
#for(i in 1:nrow(db_case)){
#  con <- DBI::dbConnect(RSQLite::SQLite(),
#                        dbname="C:/_sqlite/iljung_variable.db" ) # db파일의 경로로 수정
#  
#  
#  daily_by_minute<- read_csv( files %>% 
#                                enframe() %>%
#                                select(value) %>%
#                                filter(value %>%
#                                         str_detect(db_case$market_key[i]) &value %>% 
#                                         str_detect("일중") &value %>%
#                                         str_detect(db_case$yearmonth[i]) ) %>% pull
#                              ,locale =locale,col_select = c(1,2,5,9) )
#  
#  
#  daily_by_minute<- inner_join(daily_by_minute,n_day_data, by = c("거래일자"="open_date","종목명")) %>% 
#    select(-종목명) %>% 
#    filter(!is.na(종목코드))
#  
#  
#  daily_by_minute |> 
#    fmutate(시간 = as.numeric(시각)) |> 
#    filter(between(x = 시간,left = 900,right = 1530)) |>
#    fgroup_by(종목코드, 거래일자) |>
#    fmutate(수익률 = (종가-종가_lag)/종가_lag) |>
#    fsummarise(일중수익률변동성 = sd(수익률,na.rm = TRUE))->daily_by_minute
#  
#  
#  #DBI::dbCreateTable(con,"iljung",daily_by_minute) # 최초1회 (틀 만들기)
#  dbWriteTable(con, "iljung",daily_by_minute , row.names = FALSE, append=TRUE)  
#  dbDisconnect(con)
#  rm(daily_by_minute)
#  gc()
#  print(i)
#}
#
#tictoc::toc()
con <- DBI::dbConnect(RSQLite::SQLite(),
                      dbname="C:/_sqlite/iljung_variable.db" ) # db파일의 경로로 수정

new_variable<- tbl(con,"iljung") %>% collect() # db에 저장된  거래일자별,종목코드별 일중수익률 변동성 
new_variable %>% head()
#new_variable %>% saveRDS("new_variable_daily_fluc.rds") # rds파일로 저장뒤 불러오기 

종목코드,거래일자,일중수익률변동성
<chr>,<dbl>,<dbl>
000020,20190701,0.002745859
000020,20190702,0.002286267
000020,20190703,0.003645282
000020,20190704,0.004346610
000020,20190705,0.005612956
000020,20190708,0.010730855


### 수정주가 계산 및 기존 데이터와 결합

In [ ]:
#tk_make_timeseries(start_date = "2019-07-01",end_date = "2022-06-01") %>% enframe %>% 
#  mutate(yearmonth = paste0(substr(value,1,4),substr(value,6,7))) %>% 
#  group_by(yearmonth) %>% 
#  group_keys()->date_key
#
#
#db_case<- expand_grid(market_key = c("유가증권","코스닥"),
#                      date_key)
#
#
######
## 일별 코스피 코스닥 데이터 불러와서 결합.
#
#daily_trade<-bind_rows(
#  
#  read_csv(file = files |> 
#      enframe()  |> 
#      select(value) |>  
#      filter(value |>  str_detect("코스닥") &value |>  str_detect("일별") ) |> pull()
#    ,locale =locale, num_threads = 15,col_select = c(1:9,12,19,21) ),
#  
#  read_csv( file = files |> 
#      enframe()  |> 
#      select(value) |>  
#      filter(value |>  str_detect("유가증권") &value |>  str_detect("일별") ) |> pull()
#    ,locale =locale, num_threads = 15,col_select = c(1:9,12,19,21) )
#)
#
##rev(cumprod(rev(k))) 
##cumsum(k)
#
##수정주가가 필요한 경우: 유상증자, 무상증자, 배당락, 액면분할,액면병합, 기업합병, 감자, 권리락
#
#daily_trade |> 
#  arrange(거래일자) |> # 거래일자 순서대로 정렬되어있지 않아서 정렱
#  group_by(종목코드) |> 
#  mutate(기준가_lead = lead(기준가,1), # 종목코드별로, 다음날 
#            기준가_lead = ifelse(is.na(기준가_lead),종가,기준가_lead)) |> 
#  filter(거래일자<=20220701) |> 
#  mutate(change_event = 종가 != 기준가_lead , #다음날 기준가가 그날의 종가와 같지 않다면, 수정주가 조정이 필요한 것.
#         change_ratio = 기준가_lead/종가,     #얼마나 조정되었는지 비율 확인. 액면분할을 주당5주로 하더라도, tick size가 다르기 때문에 정확히 나눠떨어지지 않음.
#         n_change = cumsum(change_event)) |>  # 종목당 주가수정이 몇번이나 필요한지 체크하기 위한 변수.
#  arrange(-거래일자) |>  # 수정주가는 최근에서부터 거슬러 올라가면서 조정해 줘야하기 때문에 최근날짜순으로 조정
#  mutate(cprod = cumprod(change_ratio), # 바뀐비율의 누적곱
#         constant = cprod / change_ratio) |>  # 매일매일의 데이터에 곱해줘야 하는 상수
#  mutate(constant = ifelse(change_ratio !=1, constant*change_ratio,constant)) |> 
#  ungroup() |> 
#  mutate(across(.cols = c(시가,고가,저가,종가),.fns = ~.*constant ), # 가격에 상수 곱해주기
#         거래량 = 거래량/constant) |>  # 거래량도 수정 거래량.
#  arrange(거래일자) |> 
#  select(거래일자,종목코드,종목명,시가,고가,저가,종가,거래량)->adjusted_price
#
## 7월 마지막을 기준으로 수정주가 적용완료. 새로 이벤트가 생기면 그때마다 적용해줘야함.(추후 자동화시켜야함.)
#
#inspectdf::inspect_na(adjusted_price)
#
#
#adjusted_price
#
#n_day_data<- fread("C:/Users/shj/Documents/2022_summer_project/krx_repo/trading_2019_to_2022.csv")
#n_day_data %>%
#  mutate(종목코드 = sprintf("%06d",종목코드) ) |> 
#  left_join(readRDS(file = "new_variable_daily_fluc.rds"),
#            by = c("종목코드","open_date"="거래일자")) ->n_day_data
## 
## adjusted_price |>  
##   rename(open_date=거래일자)->adjusted_price
#
#keys <- c("open_date","종목명")
#n_day_data
#
## 생각해보니 거래량, 상장주식수도 수정이 필요함. 하지만 필요한 경우와 필요하지 않은 경우로 나뉘는데,(배당락, 권리락에 대해 상장주식수 수정은 필요 x)
# 현재 거래소 데이터에 있는 `락구분`(배당락,권리락, 액면분할 병합, 감자 등 으로 추정) 변수가 무엇을 의미하는지 알 수 없어 따로 수정하지 않음. 
#
#for(i in 1:nrow(db_case)){
#  
#  con <- DBI::dbConnect(RSQLite::SQLite(),
#                        dbname="C:/_sqlite/daily_and_iljung.db" ) # db파일의 경로로 수정
#  
#  
#  daily_by_minute<- read_csv( files %>% 
#                                enframe() %>%
#                                select(value) %>%
#                                filter(value %>%
#                                         str_detect(db_case$market_key[i]) &value %>% 
#                                         str_detect("일중") &value %>%
#                                         str_detect(db_case$yearmonth[i]) ) %>% pull,locale =locale,col_select = c(1,4,5) ) |> distinct()
#  
#  
#  daily_by_minute<- inner_join(daily_by_minute,adjusted_price, by = c("거래일자","종목코드")) %>% 
#    select(-종목명_delete) %>% 
#    filter(!is.na(종목코드))
#  
#
#  #DBI::dbCreateTable(con,"iljung",daily_by_minute) # 최초1회 (틀 만들기)
#  dbWriteTable(con, "ticker_name",daily_by_minute , row.names = FALSE, append=TRUE)  
#  dbDisconnect(con)
#  rm(daily_by_minute)
#  gc()
#  print(i)
#}
#
#con <- DBI::dbConnect(RSQLite::SQLite(),
#                      dbname="C:/_sqlite/daily_and_iljung.db" ) # db파일의 경로로 수정
#
#ticker_name<- tbl(con,"ticker_name") %>% collect()
#
#
#ticker_name |> 
#  rename(open_date = 거래일자) |> 
#  select(-종목코드)->ticker_name
#
#
#adjusted_n_day_data<- n_day_data |>
#  select(-c("시가","고가","저가","종가","거래량")) |> 
#  left_join(ticker_name, keys) |> 
#  filter(open_date>=20190701)
#
#
#adjusted_n_day_data %>% tail
#adjusted_n_day_data |> saveRDS("new_variable_with_adjusted_price.rds")


adjusted_n_day_data  <- read_rds("C:/Users/shj/Documents/2022_summer_project/krx_repo/new_variable_with_adjusted_price.rds")
adjusted_n_day_data |> 
  filter(종목명 == "카카오")


# 따라서 이 파일이 기존의 trading_2019_2022.csv에 일중데이터 결합 및 수정주가를 결합한 것이므로 이파일을 통해서 기존 작업을 진행하면 된다.



종목코드,종목명,시장구분,소속부,대비,등락률,거래대금,시가총액,상장주식수,상장일,⋯,외국인한도수량,외국인한도소진율,업종명,open_date,일중수익률변동성,시가,고가,저가,종가,거래량
<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<int64>,<int64>,<int64>,<date>,⋯,<int64>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
035720,카카오,KOSPI,NA,-2000,-1.52,56346257000,10805787303500,83442373,2017-07-10,⋯,83442373,25.19,서비스업,20190701,0.003974925,26594.98,26695.34,25892.47,25992.83,2160929.7
035720,카카오,KOSPI,NA,2000,1.54,34393070000,10972672049500,83442373,2017-07-10,⋯,83442373,25.23,서비스업,20190702,0.006218798,25992.83,26394.27,25691.76,26394.27,1317717.0
035720,카카오,KOSPI,NA,-500,-0.38,39313671000,10930950863000,83442373,2017-07-10,⋯,83442373,25.25,서비스업,20190703,0.003452820,26394.27,26494.62,26093.19,26293.91,1497965.9
035720,카카오,KOSPI,NA,1500,1.15,47631627500,11056114422500,83442373,2017-07-10,⋯,83442373,25.27,서비스업,20190704,0.005257089,26293.91,26594.98,25992.83,26594.98,1806694.4
035720,카카오,KOSPI,NA,1000,0.75,61093675000,11139556795500,83442373,2017-07-10,⋯,83442373,25.22,서비스업,20190705,0.005320615,26695.34,26896.06,26394.27,26795.70,2283899.0
035720,카카오,KOSPI,NA,-5500,-4.12,75587455500,10680623744000,83442373,2017-07-10,⋯,83442373,25.25,서비스업,20190708,0.009021897,26795.70,26795.70,25691.76,25691.76,2894625.0
035720,카카오,KOSPI,NA,3000,2.34,60621048500,10930950863000,83442373,2017-07-10,⋯,83442373,25.17,서비스업,20190709,0.004108085,25792.11,26494.62,25691.76,26293.91,2309920.7
035720,카카오,KOSPI,NA,0,0.00,41006896000,10930950863000,83442373,2017-07-10,⋯,83442373,25.27,서비스업,20190710,0.004534924,26494.62,26494.62,25892.47,26293.91,1559126.7
035720,카카오,KOSPI,NA,-500,-0.38,68876461000,10891369876500,83458773,2017-07-10,⋯,83458773,25.40,서비스업,20190711,0.003176805,26293.91,26695.34,26193.55,26193.55,2620198.6


### KOSPI/KOSDAQ 지수 데이터 가져오기

In [ ]:
# 한국은행 api 에서 행 번호 23,24번이 코스피,코스닥 일별 시장지수
purpose<- bind_cols(keyStatList, stat_code= stat_code,item_code1=item_code1) %>% slice(c(23,24))


purpose %>% 
  mutate(cycle_code = cycle_to_code(cycle)) -> purpose

purpose<- left_join(purpose,cycle_time_form ,by = c("cycle_code"="cycle"))

all_day<- tibble(time = tk_make_timeseries(start_date = "2019-01-01",end_date = "2022-06-30"))
KOSPI_KOSDAQ <- all_day # 최종 저장할 tibble(=데이터프레임)

for( i in 1:nrow(purpose)){
  parse_code<- paste0("ecos::statSearch(my_key,"           ,
                      "stat_code ='" ,  purpose$stat_code[i],
                      "',item_code1 ='", purpose$item_code1[i],
                      "',cycle ='",      purpose$cycle_code[i],
                      "',start_time = '",purpose$start_time[i],
                      "',end_time ='", purpose$end_time[i], "')")
  
  # 데이터 읽기.
  data <- eval(parse(text = parse_code)) %>%  select(tail(names(.), 3))  
  colnames(data)<- c("time",paste0(purpose$keystat_name[i],"(",data$unit_name[1],")"),"unit_name")
  data %>% 
    select(head(names(.),2)) ->data
  
  
  data <- left_join(all_day,data %>% mutate(time = ymd(time)), by = "time")
  
  KOSPI_KOSDAQ <- left_join(KOSPI_KOSDAQ,data,by = "time")
  
}

KOSPI_KOSDAQ %>% head(20)

Using by: day



time,코스피지수(1980.01.04=100),코스닥지수(1996.07.01=1000)
<date>,<dbl>,<dbl>
2019-01-01,NA,NA
2019-01-02,2010.00,669.37
2019-01-03,1993.70,657.02
2019-01-04,2010.25,664.49
2019-01-05,NA,NA
2019-01-06,NA,NA
2019-01-07,2037.10,672.84
2019-01-08,2025.27,668.49
2019-01-09,2064.71,679.74


### 3년치 일중 데이터 SQlite database에 저장

In [ ]:
#library(DBI)
#library(RSQLite)
##library(odbc)
#library(data.table)
#library(tidyverse)
#library(readr)
##library(dbplyr)
#library(lubridate)
#library(timetk)
#tk_make_timeseries(start_date = "2019-07-01",end_date = "2022-07-01") %>% enframe %>% 
#  mutate(yearmonth = paste0(substr(value,1,4),substr(value,6,7))) %>% 
#  group_by(yearmonth) %>% 
#  group_keys()->date_key
#
#
#n_day_data<- fread("C:/Users/shj/Documents/2022_summer_project/krx_repo/trading_2019_to_2022.csv",encoding = "UTF-8")
#n_day_data %>% 
#  filter(open_date>=20190701) %>% 
#  mutate(종목코드 = sprintf("%06d",종목코드) ) %>% 
#  select(open_date, 종목명,종목코드) ->n_day_data
#
#
#db_case<- expand_grid(market_key = c("유가증권","코스닥"),
#            date_key)
#
#
#
##현재의 작업공간에 있는 csv 파일의 목록 가져오기(recursive = T 로 하위폴더 포함)
#files <-  dir(pattern="csv",recursive = T)
#encoding<- readr::guess_encoding(files[123]) %>% slice(1) %>% pull(encoding)
#locale <- readr::locale(encoding = encoding)
#
#
#tictoc::tic()
#
#
#for(i in 1:nrow(db_case)){
#  con <- DBI::dbConnect(RSQLite::SQLite(),
#                        dbname="C:/_sqlite/krx_2019_2022.db" ) # db파일의 경로로 수정
#  
#  
#  daily_by_minute<- read_csv( files %>% 
#                                enframe() %>%
#                                select(value) %>%
#                                filter(value %>%
#                                         str_detect(db_case$market_key[i]) &value %>% 
#                                         str_detect("일중") &value %>%
#                                         str_detect(db_case$yearmonth[i]) ) %>% pull
#                             ,locale =locale, num_threads = parallel::detectCores()-1 ,col_select = c(1,2,5,6:9) )
#  
#  
#  daily_by_minute<- left_join(daily_by_minute,n_day_data, by = c("거래일자"="open_date","종목명")) %>% 
#    select(-종목명)
#  # DBI::dbCreateTable(con,"iljung",daily_by_minute) # 최초1회 (틀 만들기)
#  dbWriteTable(con, "iljung",daily_by_minute , row.names = FALSE, append=TRUE)  
#  dbDisconnect(con)
#  rm(daily_by_minute)
#  print(i)
#}
#
#tictoc::toc()
#3시18분 시작 
#3시58분 끝


#krx_2019_2022.db 파일 생성. 추후에 이것도 수정주가를 고려하여 반영한 뒤 저장해야함.

### 거시변수 지표 활용의 유의성 검정


*   탈출률 차이에 관한 검정
    1. 백테스팅 툴에 활용한 거시변수 8개에 대하여 각 거시변수별로 거시변수의 모멘텀이 +인 날짜와 -인 날짜로 전체기간의 투자시작일을 나눕니다
    2. 거시변수 모멘텀이 +인 투자시작일의 탈출률과 -인 투자시작일의 탈출률을 벡터 형태로 반환합니다
    3. 거시변수 모멘텀이 +인 투자시작일의 탈출률과 -인 투자시작일의 탈출률이 유의미한 차이가 있는지 Welch Two Sample t-test를 활용하여 5% 유의수준에서 유의성 검정합니다


*   수익률 차이에 관한 검정
    1. 백테스팅 툴에 활용한 거시변수 8개에 대하여 각 거시변수별로 거시변수의 모멘텀이 +인 날짜와 -인 날짜로 전체기간의 투자시작일을 나눕니다
    2. 거시변수 모멘텀이 +인 투자시작일의 수익률과 -인 투자시작일의 수익률을 벡터 형태로 반환합니다
    3. 거시변수 모멘텀이 +인 투자시작일의 수익률과 -인 투자시작일의 수익률이 유의미한 차이가 있는지 Welch Two Sample t-test를 활용하여 5% 유의수준에서 유의성 검정합니다






In [ ]:
# _1. Package load ------------------------------------------------------------
library(gt)

# _2. Data ,code load ---------------------------------------------------------------

future::plan(workers = parallel::detectCores()-1,strategy = "multisession")


n_day_data<- fread("E:\\trading_2019_to_2021.csv", encoding = 'UTF-8')
n_day_data %>% 
    fmutate(종목코드 = sprintf("%06d",종목코드) ) %>% 
    left_join(readRDS(file = "E:\\new_variable_daily_fluc.rds"),
              by = c("종목코드","open_date"="거래일자")) %>% 
    fmutate(open_date = ymd(open_date)) ->n_day_data
#source("C:/Users/shj/Documents/2022_summer_project/krx_repo/0366_seongho/03_scripts/back_test_function.R")



# 스팩주 두개가 이름에 스팩없는상태여서 필터링이 안됐음. 이거 삭제해야할듯.
# 추가적으로 투자자보호를 위해 관리종목, 투자주의 환기종목은 필터링에서 제외할까?
# n_day_data %>% 
#   filter(!(소속부 %in% c("관리종목(소속부없음)","SPAC(소속부없음)")))

macro_filter <- fread("E:\\macro_filter.csv", encoding = 'UTF-8')
esg.data <- read_csv("E:\\esg_table.csv")
esg.data %>% 
    select(2,4) %>%
    filter(nchar(ESG등급)>0)->esg.data

# ESG 합치기
n_day_data <- merge.data.table(n_day_data, esg.data, by.x="종목명", by.y="기업명", all.x=T)

n_day_data %<>% 
    fmutate(ESG등급 = ifelse(is.na(ESG등급),"미지정",ESG등급) %>%as_factor()) %>% 
    fmutate(ESG등급 = ESG등급 %>% fct_relevel("A+","A","B+","B","C","D","미지정") )

n_day_data$ESG등급 %>% levels()
# ECOS 데이터 합치기

n_day_data <-merge.data.table(n_day_data, macro_filter, by.x="open_date", by.y="time", all.x=T)




# _3. 필터링을 위한 변수 계산 ------------------------------------------------------------------
return_rank_percentile<- function(x,invesrse = FALSE){
    if(invesrse == TRUE){
        x <- -x
    }
    x <- (frank(x,ties.method ="min")/length(x))*100 # 작은값일수록 순위가 높음. (= 최소값이 1등)
    return(x)
}



n_day_data |>
    # 변수만드는데 필요한 계산 먼저 완료.
    fmutate(시가총액 = 시가총액 / 100000000)  |>  #overflow문제있는것같아서 억단위로 바꿈.영향 없음.
    fgroup_by(종목코드)  |> 
    fmutate(거래량_lag = flag(거래량,1))  |> 
    fmutate(고가_lag = flag(고가, 1))  |> 
    fmutate(roll_mean_30_거래= frollmean(x = 거래량_lag,n = 30,align = "right",fill = 1) )  |>  
    fmutate(변동성= 거래량/roll_mean_30_거래)  |>  
    fmutate(`최근20거래일고가` = frollapply(고가_lag, 20, max))  |> 
    fmutate(`20거래일고가대비등락률` = (종가-최근20거래일고가)/종가) |>  
    fungroup() |>
    fgroup_by(open_date) |>
    fmutate(new_PER = ifelse(is.na(PER),0,1/PER)) |>
    fmutate(유동성 = 거래량/상장주식수) |>
    
    # 계산된 변수들에 percentile 반환하는 함수 일괄 적용
    fmutate(across(.cols = c(시가총액,new_PER,유동성,변동성, `20거래일고가대비등락률`,일중수익률변동성),
                   .fns = function(x)return_rank_percentile(x,invesrse = TRUE),
                   .names = function(.cols,f) paste0(.cols,"_percentile") )) |>
    fungroup() %>%
    fselect(names(.) %>% grep(pattern ="open_date|종목|종가|거래량|소속부|시장구분|momentum|_percentile|ESG")) |>
    # 앞에 NA 자르고, 2022-06-30 까지 있으니까 현재 기준으로 필터링은 되지만 아직 탈출률 결정안된 날들이 있음
    # 일중데이터를 통한 보완은 1분기밖에 없어서 못했다고하면 될듯.
    # 깔끔하게 2019-07-01 ~ 2022-06-30 3년데이터
    filter(open_date >="2019-07-01") ->  processed_data

#processed_data %>% write_csv("processed_data.csv")




# 그리드서치 함수-------------------------------------------------------------------------------------------

result_back_test <- list()

result_back_test[[1]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.1,minus=-0.03,a=0,obs_period=15.rds")
result_back_test[[2]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.1,minus=-0.03,a=0,obs_period=30.rds")
result_back_test[[3]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.1,minus=-0.03,a=-0.01,obs_period=15.rds")
result_back_test[[4]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.1,minus=-0.03,a=-0.01,obs_period=30.rds")
result_back_test[[5]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.1,minus=-0.03,a=-0.02,obs_period=15.rds")
result_back_test[[6]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.1,minus=-0.03,a=-0.02,obs_period=30.rds")
result_back_test[[7]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.1,minus=-0.05,a=0,obs_period=15.rds")
result_back_test[[8]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.1,minus=-0.05,a=0,obs_period=30.rds")

gc()

result_back_test[[9]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.1,minus=-0.05,a=-0.01,obs_period=15.rds")
result_back_test[[10]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.1,minus=-0.05,a=-0.01,obs_period=30.rds")
result_back_test[[11]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.1,minus=-0.05,a=-0.02,obs_period=15.rds")
result_back_test[[12]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.1,minus=-0.05,a=-0.02,obs_period=30.rds")
result_back_test[[13]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.1,minus=-0.07,a=0,obs_period=15.rds")
result_back_test[[14]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.1,minus=-0.07,a=0,obs_period=30.rds")
result_back_test[[15]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.1,minus=-0.07,a=-0.01,obs_period=15.rds")
result_back_test[[16]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.1,minus=-0.07,a=-0.01,obs_period=30.rds")
result_back_test[[17]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.1,minus=-0.07,a=-0.02,obs_period=15.rds")
result_back_test[[18]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.1,minus=-0.07,a=-0.02,obs_period=30.rds")

gc()

result_back_test[[19]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.05,minus=-0.03,a=0,obs_period=15.rds")
result_back_test[[20]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.05,minus=-0.03,a=0,obs_period=30.rds")
result_back_test[[21]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.05,minus=-0.03,a=-0.01,obs_period=15.rds")
result_back_test[[22]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.05,minus=-0.03,a=-0.01,obs_period=30.rds")
result_back_test[[23]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.05,minus=-0.03,a=-0.02,obs_period=15.rds")
result_back_test[[24]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.05,minus=-0.03,a=-0.02,obs_period=30.rds")

gc()

result_back_test[[25]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.07,minus=-0.03,a=0,obs_period=15.rds")
result_back_test[[26]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.07,minus=-0.03,a=0,obs_period=30.rds")
result_back_test[[27]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.07,minus=-0.03,a=-0.01,obs_period=15.rds")
result_back_test[[28]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.07,minus=-0.03,a=-0.01,obs_period=30.rds")
result_back_test[[29]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.07,minus=-0.03,a=-0.02,obs_period=15.rds")
result_back_test[[30]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.07,minus=-0.03,a=-0.02,obs_period=30.rds")
result_back_test[[31]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.07,minus=-0.05,a=0,obs_period=15.rds")
result_back_test[[32]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.07,minus=-0.05,a=0,obs_period=30.rds")
result_back_test[[33]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.07,minus=-0.05,a=-0.01,obs_period=15.rds")
result_back_test[[34]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.07,minus=-0.05,a=-0.01,obs_period=30.rds")
result_back_test[[35]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.07,minus=-0.05,a=-0.02,obs_period=15.rds")
result_back_test[[36]] <- readRDS(file = "E:\\back_test_results\\back_test_chance=0.07,minus=-0.05,a=-0.02,obs_period=30.rds")


gc()

# 필터링을 위해 최소한 거래량이 있는 종목을 선정한다. 또한, 관리종목대상으로는 투자하지 않기로한다.
processed_data |> 
    filter(거래량>0) |> 
    filter(!(소속부 %in% c("관리종목(소속부없음)","투자주의환기종목(소속부없음)","SPAC(소속부없음)"))) ->
    processed_data_modify


# 거시변수 그리드 만들기(macro_grid)
macro_grid <- list( list(1, c(-1,1), c(-1,1), c(-1,1), c(-1,1), c(-1,1), c(-1,1), c(-1,1)),
                    list(-1, c(-1,1), c(-1,1), c(-1,1), c(-1,1), c(-1,1), c(-1,1) ,c(-1,1)),
                    
                    list(c(-1,1), 1, c(-1,1), c(-1,1), c(-1,1), c(-1,1), c(-1,1) ,c(-1,1)),
                    list(c(-1,1), -1, c(-1,1), c(-1,1), c(-1,1), c(-1,1), c(-1,1) ,c(-1,1)),
                    
                    list(c(-1,1), c(-1,1), 1, c(-1,1), c(-1,1), c(-1,1), c(-1,1) ,c(-1,1)),
                    list(c(-1,1), c(-1,1), -1, c(-1,1), c(-1,1), c(-1,1), c(-1,1) ,c(-1,1)),
                    
                    list(c(-1,1), c(-1,1), c(-1,1), 1, c(-1,1), c(-1,1), c(-1,1) ,c(-1,1)),
                    list(c(-1,1), c(-1,1), c(-1,1), -1, c(-1,1), c(-1,1), c(-1,1) ,c(-1,1)),
                    
                    list(c(-1,1), c(-1,1), c(-1,1), c(-1,1), 1, c(-1,1), c(-1,1) ,c(-1,1)),
                    list(c(-1,1), c(-1,1), c(-1,1), c(-1,1), -1, c(-1,1), c(-1,1) ,c(-1,1)),
                    
                    list(c(-1,1), c(-1,1), c(-1,1), c(-1,1), c(-1,1), 1, c(-1,1) ,c(-1,1)),
                    list(c(-1,1), c(-1,1), c(-1,1), c(-1,1), c(-1,1), -1, c(-1,1) ,c(-1,1)),
                    
                    list(c(-1,1), c(-1,1), c(-1,1), c(-1,1), c(-1,1), c(-1,1), 1 ,c(-1,1)),
                    list(c(-1,1), c(-1,1), c(-1,1), c(-1,1), c(-1,1), c(-1,1), -1 ,c(-1,1)),
                    
                    list(c(-1,1), c(-1,1), c(-1,1), c(-1,1), c(-1,1), c(-1,1), c(-1,1), 1),
                    list(c(-1,1), c(-1,1), c(-1,1), c(-1,1), c(-1,1), c(-1,1), c(-1,1), -1))


expand_grid(
    chance = c(0.1,0.07,0.05,0.03),
    minus = -c(0.1,0.07,0.05,0.03),
    a= c(0,-0.01,-0.02),
    obs_period = c(15,30)
) |> filter(abs(chance)>abs(minus)) -> chance_grid



# 결과값 저장할 데이터 프레임 생성
result_t_test_escape <- data.frame(matrix(0, ncol = (length(macro_grid)/2+1), nrow = 37))
colnames(result_t_test_escape) <- c("매수-매도조건"," 콜금리", "회사채수익률", "달러환율", "물가", "유가", "뉴스", "위안환율", "금")

result_t_test_return <- data.frame(matrix(0, ncol = (length(macro_grid)/2+1), nrow = 37))
colnames(result_t_test_return) <- c("매수-매도조건","콜금리", "회사채수익률", "달러환율", "물가", "유가", "뉴스", "위안환율", "금")

for(i in 1:nrow(chance_grid)){
    result_t_test_escape[i,1] <- str_glue("목표수익률={chance_grid$chance[i]*100}%, 최대손실률={chance_grid$minus[i]*100}%, 목표구매가={chance_grid$a[i]*100}%, 관찰기준={chance_grid$obs_period[i]}일")
    result_t_test_return[i,1] <- str_glue("목표수익률={chance_grid$chance[i]*100}%, 최대손실률={chance_grid$minus[i]*100}%, 목표구매가={chance_grid$a[i]*100}%, 관찰기준={chance_grid$obs_period[i]}일")
}
    
result_t_test_escape[37,1] <- "전체 케이스 중 탈출률 차이가 유의한 케이스의 비율"
result_t_test_return[37,1] <- "전체 케이스 중 수익률 차이가 유의한 케이스의 비율"


for(k in 1:36){
    
    print(k)
    
    gc()
    
    result_back_test_temp <- list()
    result_back_test_temp <- result_back_test[[k]]
    
    # 필터링을 위해 최소한 거래량이 있는 종목을 선정한다. 또한, 관리종목대상으로는 투자하지 않기로한다.
    processed_data_modify_temp <- list()
    processed_data_modify_temp <- left_join(processed_data_modify,result_back_test_temp %>%
                                          select(filtered_date,종목코드,contains("first"),contains("실제"),final_price,구매),
                                      by=c("open_date"="filtered_date","종목코드"))
    
    
    # ecos_filtered_date에 거시변수 필터링에 따른 날짜 저장
    ecos_filtered_data <- list()
    
    
    for (i in 1:length(macro_grid)){
        
        # 그리드 적용 위한 변수 설정
        금리 = macro_grid[[i]][[1]]
        회사채 = macro_grid[[i]][[2]]
        달러환율 = macro_grid[[i]][[3]]
        물가 = macro_grid[[i]][[4]]
        유가 = macro_grid[[i]][[5]]
        뉴스 = macro_grid[[i]][[6]]
        위안환율 = macro_grid[[i]][[7]]
        금 = macro_grid[[i]][[8]]
        
        
        processed_data_modify_temp |> 
            #____확인하고 싶은 날짜 필터링----------------------
        filter(open_date %>% between(lower = "2019-07-01", upper = "2022-06-30")) %>%  
            {.->> selected_date} %>% 
            filter(`콜금리(익일물)(연%) _momentum` %in% 금리) %>%
            filter(`회사채수익률(3년,AA-)(연%) _momentum` %in% 회사채) %>% 
            filter(`원/달러 환율(종가)(원) _momentum` %in% 달러환율) %>%  
            filter(`소비자물가지수(2020=100) _momentum` %in% 물가) %>%  
            filter(`국제유가(Dubai, 현물)(U$/bbl) _momentum` %in% 유가) %>%  
            filter(`뉴스심리지수() _momentum` %in% 뉴스) %>%
            filter(`원/위안 환율(종가)(원) _momentum` %in% 위안환율) %>%
            filter(`금($/OZ) _momentum` %in% 금) %>%
            select(open_date, 종목코드, 시장구분, contains("percentile"),
                   contains("first"), contains("실제"),final_price, 구매) -> ecos_filtered_data[[i]] 
        
         gc()
    }
    
    #탈출률 유의성
    for (j in 1:(length(macro_grid)/2)){
        
        # default 값
        result_grid_search_plus <- list()
        result_grid_search_minus <- list()
        
        
        ecos_filtered_data[[(2*j-1)]] |>
            fselect(종목코드, 탈출_first) |>
            fmutate(탈출여부 = ifelse(탈출_first == 10000, 0, 1)) |>
            fselect(탈출여부)-> result_grid_search_plus
        # 탈출여부는 탈출했으면 1, 탈출실패했으면 0
        
        ecos_filtered_data[[2*j]] |>
            fselect(종목코드, 탈출_first) |>
            fmutate(탈출여부 = ifelse(탈출_first == 10000, 0, 1))  |>
            fselect(탈출여부)-> result_grid_search_minus
        
        result_grid_search_plus <- unlist(result_grid_search_plus,use.names=FALSE)
        
        result_grid_search_minus <- unlist(result_grid_search_minus,use.names=FALSE)
        
        if(length(result_grid_search_plus)==0 | length(result_grid_search_minus)==0 ){
            result_t_test_escape[k,j+1] <- "티커없음"  
        }else{
            t_test <- t.test(result_grid_search_plus, result_grid_search_minus)$p.value
            
            if(t_test<0.05){
                result_t_test_escape[k,j+1] <- "유의"  
            }else{
                result_t_test_escape[k,j+1] <- "유의하지 않음"                 
            }
 
        }
        gc()
     }
    
    # 수익률 유의성
    for (m in 1:(length(macro_grid)/2)){
        # default 값
        result_grid_search_plus <- list()
        result_grid_search_minus <- list()
        
        
        ecos_filtered_data[[(2*m-1)]] |>
            fmutate(ending_position = fifelse(탈출_first==10000 &손절_first==10000, "최종가매도",
                                                 if_else(탈출_first !=10000,"탈출","손절")) |>as_factor()) |>
            fmutate(ending_return = fifelse(ending_position =="최종가매도",
                                            final_price/실제구매가-1 ,
                                            if_else(ending_position=="탈출",
                                                    실제익절가_first/실제구매가-1,
                                                    실제손절가_first/실제구매가-1))) |>
            filter(구매==1) |>
            select(ending_return) -> result_grid_search_plus
        
        
        ecos_filtered_data[[2*m]] |>
            fmutate(ending_position = fifelse(탈출_first==10000 &손절_first==10000, "최종가매도",
                                                 if_else(탈출_first !=10000,"탈출","손절")) |>as_factor()) |>
            fmutate(ending_return = fifelse(ending_position =="최종가매도",
                                            final_price/실제구매가-1 ,
                                            if_else(ending_position=="탈출",
                                                    실제익절가_first/실제구매가-1,
                                                    실제손절가_first/실제구매가-1))) |>
            filter(구매==1) |>
            select(ending_return) -> result_grid_search_minus
        
        result_grid_search_plus <- unlist(result_grid_search_plus,use.names=FALSE)
        
        result_grid_search_minus <- unlist(result_grid_search_minus,use.names=FALSE)
        
        if(length(result_grid_search_plus)==0 | length(result_grid_search_minus)==0 ){
            result_t_test_return[k,m+1] <- "티커없음"  
        }else{
            t_test <- t.test(result_grid_search_plus, result_grid_search_minus)$p.value
            
            if(t_test<0.05){
                result_t_test_return[k,m+1] <- "유의"  
            }else{
                result_t_test_return[k,m+1] <- "유의하지 않음"                 
            }
            
        }
        gc() 
     }
    
}



result_t_test_escape[37,2:9] <- c(round(length(which(result_t_test_escape[1:36,2]=="유의"))/36,3),
                                        round(length(which(result_t_test_escape[1:36,3]=="유의"))/36,3),
                                        round(length(which(result_t_test_escape[1:36,4]=="유의"))/36,3),
                                        round(length(which(result_t_test_escape[1:36,5]=="유의"))/36,3),
                                        round(length(which(result_t_test_escape[1:36,6]=="유의"))/36,3),
                                        round(length(which(result_t_test_escape[1:36,7]=="유의"))/36,3),
                                        round(length(which(result_t_test_escape[1:36,8]=="유의"))/36,3),
                                        round(length(which(result_t_test_escape[1:36,9]=="유의"))/36,3))


result_t_test_return[37,2:9] <- c(round(length(which(result_t_test_return[1:36,2]=="유의"))/36,3),
                                        round(length(which(result_t_test_return[1:36,3]=="유의"))/36,3),
                                        round(length(which(result_t_test_return[1:36,4]=="유의"))/36,3),
                                        round(length(which(result_t_test_return[1:36,5]=="유의"))/36,3),
                                        round(length(which(result_t_test_return[1:36,6]=="유의"))/36,3),
                                        round(length(which(result_t_test_return[1:36,7]=="유의"))/36,3),
                                        round(length(which(result_t_test_return[1:36,8]=="유의"))/36,3),
                                        round(length(which(result_t_test_return[1:36,9]=="유의"))/36,3))



# gt package 활용해서 표로 출력
result_t_test_escape %>% 
    gt() %>%
    tab_header(title = "거시변수 모멘텀 조건에 따른 탈출률 차이가 유의한지 유의수준 5%에서 검정")

result_t_test_return %>% 
    gt() %>%
    tab_header(title = "거시변수 모멘텀 조건에 따른 수익률 차이가 유의한지 유의수준 5%에서 검정")


# 그리드 서치 함수 끝----------------------------------------------------------------------------------------

**검정 결과**



1.   탈출률에 대한 유의성 검정 결과표
<p align="center"> <img src = https://drive.google.com/uc?id=1h9zarIgca-Q4n5peOF1N6begazG4YcwH>


2.   수익률에 대한 유의성 검정 결과표
<p align="center"> <img src = https://drive.google.com/uc?id=1XNGrvBnyDcNdWFO8C1gVeY1uMh31cBaP>

### 참고문헌

한국은행 100대 통계지표 중 백테스팅에 사용할 경제지표들을 선정하기 위해 주식시장과 거시경제변수의 연관성을 다룬 논문들을 참고하였습니다. 아래의 논문들에서 공통적으로 사용된 변수들 위주로 선정하였으며, 그 중 초보 투자자들도 직관적으로 뜻을 이해할 수 있는 5개의 경제지표들 최종적으로 결정하였습니다. 참고한 논문은 다음과 같습니다.



*   구진웅, 강상훈 (2020), "거시경제 불확실성이 주식시장 고유변동성에 미치는 영향 분석", 금융공학연구, 20(2), 21-48.
*   홍형석(2020), "거시지표와 딥러닝 알고리즘을 이용한 자동화된 주식 매매연구", 디지털융복합연구, 18(11), 267-272.
*   이재응, 한재형 (2021), "설명 가능한 KOSPI 증감 예측 딥러닝 모델을 위한 Layer-wise-Relevance Propagation 기반 기술적 지표 및 거시경제 지표 영향 분석", 정보과학회, 48(12), 1289-1297.
*   김화련, 홍승혜, 홍헬렌 (2021), "온라인 뉴스와 거시경제 지표, 금융 지표, 기술적 지표, 관심도 지표를 이용한 코스닥 상장 기업의 기계학습 기반 주가 변동 예측, 한국멀티미디어학회, 24(3), 448-459.
*   김재일, 김주일 (2014), "우리나라 주가와 거시경제변수들 간의 상호연관성에 관한 연구", 전문경영인연구, 17(3), 163-186.
*   이윤구 (2021), "주식수익률에 대한 거시경제변수의 상황별 영향", 경영컨설팅연구, 21(1), 103-119.

